In [1]:
%matplotlib inline
import pickle
import numpy as npp
import matplotlib.pyplot as plt
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from mxnet import ndarray as nd
from d2l import mxnet as d2l

from mxnet import init
from mxnet import autograd, gluon, np, npx, init
from mxnet.gluon import nn
from mxnet.gluon import loss as gloss
npx.set_np()

In [52]:
import math

In [2]:
%%time
#Load data
with open('msd_full.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1

if (doscaling == 1):
    # standardize feature values
    
    X_train_temp = np.array(msd_data['X_train'].astype(np.float32))
    X_test_temp = np.array(msd_data['X_test'].astype(np.float32))
    X_mu = np.mean(X_train_temp,axis=0)
    X_sigma = np.std(X_train_temp,axis=0)
    
    X_train_all = (X_train_temp-X_mu)/X_sigma   
    X_train = (X_train_temp-X_mu)/X_sigma
    X_test = (X_test_temp-X_mu)/X_sigma
    
#     xscaler = preprocessing.StandardScaler().fit(X_train_temp) 
#     X_train = xscaler.transform(X_train_temp)
#     X_test = xscaler.transform(X_test_temp)

else:
    X_train = np.array(msd_data['X_train'].astype(np.float32))
    X_test = np.array(msd_data['X_test'].astype(np.float32))

Y_train_all = np.array(msd_data['Y_train'].astype(np.float32))
Y_train = np.array(msd_data['Y_train'].astype(np.float32))
Y_test = np.array(msd_data['Y_test'].astype(np.float32))

Y_mu = Y_train.mean()

X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=0.1, random_state=1)

de_Y_train_all = Y_train_all - Y_mu
de_Y_train = Y_train - Y_mu
de_Y_valid = Y_valid - Y_mu
de_Y_test = Y_test - Y_mu

Wall time: 2.53 s


In [3]:
print(X_train_all.shape)
print(Y_train_all.shape)
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)

(463715, 90)
(463715,)
(417343, 90)
(417343,)
(46372, 90)
(46372,)


In [4]:
rmse_list = [0] * 10

### Q1

#### case 1

In [19]:
# case 1
# OLS	keep	10,000	Linear Regression	None

In [20]:
%%time 
reg = LinearRegression().fit(np.array(X_train[:10000]).asnumpy(), np.array(Y_train[:10000]).asnumpy())
pred = reg.predict(np.array(X_test).asnumpy())
rmse_list[0] = sqrt(mean_squared_error(Y_test ,pred))
print("RMSE = ", rmse_list[0])

RMSE =  9.570046270198604
Wall time: 3.01 s


#### case 2

In [21]:
# case 2 
# MLP_0_dm	de-mean	10,000	MLP, no hidden layers	None
# Note that in "de-mean" setting, you should compute RMSE using the de-meaned target values in the test set 
# so that the results are consistent.

In [7]:
# https://d2l.ai/chapter_linear-networks/linear-regression-concise.html
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a Gluon data loader."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

In [63]:
def train(net, features, labels, batch_size, num_epochs, lr=0.01, wd=0): # weight decay
    data_iter = load_array((features, labels), batch_size)
    loss = gloss.L2Loss()
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd})

    loss_record = [0] * num_epochs
    # net = net.cast('float64')
    for epoch in range(1, num_epochs + 1):
        for X, y in data_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        l = loss(net(features), labels)
       
        # stop the training process if the validation RMSE remains the lowest 
        # in the next 50 epochs. 
        # Do not adopt early stopping for the last case.
        
        if math.isnan(l.mean().asnumpy()):
            print('epoch %d, loss: NaN' % (epoch))
            break
            
        loss_record[epoch-1] = int(l.mean().asnumpy()*10**5)
        stop = True
        
        if epoch == 1: 
            stop = False
        
        for i in range(epoch - 50, epoch):
            if i >= 0 and loss_record[i] != loss_record[epoch-1]:
                stop = False
                break
        if stop:
            print('Early stopping\n')
            break

        print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

In [10]:
batch_size, num_epochs = 10000, 3000
features = np.array(X_train[:10000])
labels = np.array(Y_train[:10000])
delabels = np.array(de_Y_train[:10000])

In [29]:
%%time
# tune
rmse = [0] * 3
Lr = [0.01, 0.05, 0.1]
for i in range(len(Lr)):
    print(Lr[i])
    net = nn.Sequential()
    net.add(nn.Dense(1))
    net.initialize(init.Normal(sigma=0.01))
    train(net, features, delabels, batch_size=10000, num_epochs= 1000, lr = Lr[i])
    pred = net(np.array(X_valid)).reshape(1,-1)[0]
    rmse[i] = sqrt(mean_squared_error(de_Y_valid ,pred))
    print("RMSE = ", rmse[i])

0.01
epoch 1, loss: 57.271576
epoch 2, loss: 56.907001
epoch 3, loss: 56.566338
epoch 4, loss: 56.247169
epoch 5, loss: 55.947395
epoch 6, loss: 55.665150
epoch 7, loss: 55.398800
epoch 8, loss: 55.146912
epoch 9, loss: 54.908188
epoch 10, loss: 54.681507
epoch 11, loss: 54.465855
epoch 12, loss: 54.260330
epoch 13, loss: 54.064133
epoch 14, loss: 53.876537
epoch 15, loss: 53.696911
epoch 16, loss: 53.524670
epoch 17, loss: 53.359287
epoch 18, loss: 53.200298
epoch 19, loss: 53.047268
epoch 20, loss: 52.899826
epoch 21, loss: 52.757599
epoch 22, loss: 52.620289
epoch 23, loss: 52.487587
epoch 24, loss: 52.359234
epoch 25, loss: 52.234985
epoch 26, loss: 52.114613
epoch 27, loss: 51.997910
epoch 28, loss: 51.884689
epoch 29, loss: 51.774765
epoch 30, loss: 51.667988
epoch 31, loss: 51.564194
epoch 32, loss: 51.463249
epoch 33, loss: 51.365017
epoch 34, loss: 51.269382
epoch 35, loss: 51.176228
epoch 36, loss: 51.085445
epoch 37, loss: 50.996929
epoch 38, loss: 50.910595
epoch 39, loss: 

epoch 309, loss: 44.932648
epoch 310, loss: 44.926231
epoch 311, loss: 44.919849
epoch 312, loss: 44.913506
epoch 313, loss: 44.907192
epoch 314, loss: 44.900917
epoch 315, loss: 44.894680
epoch 316, loss: 44.888474
epoch 317, loss: 44.882301
epoch 318, loss: 44.876163
epoch 319, loss: 44.870060
epoch 320, loss: 44.863987
epoch 321, loss: 44.857948
epoch 322, loss: 44.851948
epoch 323, loss: 44.845974
epoch 324, loss: 44.840038
epoch 325, loss: 44.834133
epoch 326, loss: 44.828259
epoch 327, loss: 44.822418
epoch 328, loss: 44.816605
epoch 329, loss: 44.810829
epoch 330, loss: 44.805084
epoch 331, loss: 44.799370
epoch 332, loss: 44.793686
epoch 333, loss: 44.788033
epoch 334, loss: 44.782406
epoch 335, loss: 44.776817
epoch 336, loss: 44.771252
epoch 337, loss: 44.765720
epoch 338, loss: 44.760220
epoch 339, loss: 44.754745
epoch 340, loss: 44.749302
epoch 341, loss: 44.743889
epoch 342, loss: 44.738506
epoch 343, loss: 44.733150
epoch 344, loss: 44.727821
epoch 345, loss: 44.722527
e

epoch 613, loss: 43.946522
epoch 614, loss: 43.945087
epoch 615, loss: 43.943661
epoch 616, loss: 43.942245
epoch 617, loss: 43.940826
epoch 618, loss: 43.939423
epoch 619, loss: 43.938019
epoch 620, loss: 43.936626
epoch 621, loss: 43.935234
epoch 622, loss: 43.933853
epoch 623, loss: 43.932476
epoch 624, loss: 43.931107
epoch 625, loss: 43.929741
epoch 626, loss: 43.928383
epoch 627, loss: 43.927032
epoch 628, loss: 43.925686
epoch 629, loss: 43.924339
epoch 630, loss: 43.923008
epoch 631, loss: 43.921677
epoch 632, loss: 43.920357
epoch 633, loss: 43.919037
epoch 634, loss: 43.917725
epoch 635, loss: 43.916420
epoch 636, loss: 43.915119
epoch 637, loss: 43.913826
epoch 638, loss: 43.912537
epoch 639, loss: 43.911255
epoch 640, loss: 43.909973
epoch 641, loss: 43.908703
epoch 642, loss: 43.907436
epoch 643, loss: 43.906174
epoch 644, loss: 43.904922
epoch 645, loss: 43.903671
epoch 646, loss: 43.902424
epoch 647, loss: 43.901188
epoch 648, loss: 43.899952
epoch 649, loss: 43.898724
e

epoch 917, loss: 43.700436
epoch 918, loss: 43.700020
epoch 919, loss: 43.699608
epoch 920, loss: 43.699192
epoch 921, loss: 43.698780
epoch 922, loss: 43.698368
epoch 923, loss: 43.697960
epoch 924, loss: 43.697552
epoch 925, loss: 43.697147
epoch 926, loss: 43.696743
epoch 927, loss: 43.696342
epoch 928, loss: 43.695942
epoch 929, loss: 43.695541
epoch 930, loss: 43.695141
epoch 931, loss: 43.694744
epoch 932, loss: 43.694351
epoch 933, loss: 43.693954
epoch 934, loss: 43.693562
epoch 935, loss: 43.693172
epoch 936, loss: 43.692780
epoch 937, loss: 43.692394
epoch 938, loss: 43.692005
epoch 939, loss: 43.691624
epoch 940, loss: 43.691238
epoch 941, loss: 43.690853
epoch 942, loss: 43.690475
epoch 943, loss: 43.690094
epoch 944, loss: 43.689716
epoch 945, loss: 43.689339
epoch 946, loss: 43.688961
epoch 947, loss: 43.688587
epoch 948, loss: 43.688217
epoch 949, loss: 43.687843
epoch 950, loss: 43.687473
epoch 951, loss: 43.687107
epoch 952, loss: 43.686737
epoch 953, loss: 43.686371
e

epoch 224, loss: 43.639595
epoch 225, loss: 43.638588
epoch 226, loss: 43.637596
epoch 227, loss: 43.636623
epoch 228, loss: 43.635658
epoch 229, loss: 43.634716
epoch 230, loss: 43.633789
epoch 231, loss: 43.632874
epoch 232, loss: 43.631973
epoch 233, loss: 43.631092
epoch 234, loss: 43.630222
epoch 235, loss: 43.629364
epoch 236, loss: 43.628521
epoch 237, loss: 43.627693
epoch 238, loss: 43.626877
epoch 239, loss: 43.626076
epoch 240, loss: 43.625286
epoch 241, loss: 43.624508
epoch 242, loss: 43.623741
epoch 243, loss: 43.622986
epoch 244, loss: 43.622242
epoch 245, loss: 43.621513
epoch 246, loss: 43.620792
epoch 247, loss: 43.620087
epoch 248, loss: 43.619392
epoch 249, loss: 43.618702
epoch 250, loss: 43.618027
epoch 251, loss: 43.617363
epoch 252, loss: 43.616707
epoch 253, loss: 43.616062
epoch 254, loss: 43.615425
epoch 255, loss: 43.614799
epoch 256, loss: 43.614185
epoch 257, loss: 43.613575
epoch 258, loss: 43.612980
epoch 259, loss: 43.612389
epoch 260, loss: 43.611809
e

epoch 528, loss: 43.568665
epoch 529, loss: 43.568638
epoch 530, loss: 43.568607
epoch 531, loss: 43.568577
epoch 532, loss: 43.568550
epoch 533, loss: 43.568523
epoch 534, loss: 43.568493
epoch 535, loss: 43.568466
epoch 536, loss: 43.568436
epoch 537, loss: 43.568409
epoch 538, loss: 43.568382
epoch 539, loss: 43.568352
epoch 540, loss: 43.568329
epoch 541, loss: 43.568298
epoch 542, loss: 43.568275
epoch 543, loss: 43.568249
epoch 544, loss: 43.568222
epoch 545, loss: 43.568195
epoch 546, loss: 43.568172
epoch 547, loss: 43.568142
epoch 548, loss: 43.568119
epoch 549, loss: 43.568092
epoch 550, loss: 43.568069
epoch 551, loss: 43.568043
epoch 552, loss: 43.568020
epoch 553, loss: 43.567997
epoch 554, loss: 43.567970
epoch 555, loss: 43.567947
epoch 556, loss: 43.567924
epoch 557, loss: 43.567902
epoch 558, loss: 43.567879
epoch 559, loss: 43.567852
epoch 560, loss: 43.567833
epoch 561, loss: 43.567810
epoch 562, loss: 43.567783
epoch 563, loss: 43.567764
epoch 564, loss: 43.567741
e

epoch 832, loss: 43.565102
epoch 833, loss: 43.565098
epoch 834, loss: 43.565094
epoch 835, loss: 43.565090
epoch 836, loss: 43.565086
epoch 837, loss: 43.565083
epoch 838, loss: 43.565083
epoch 839, loss: 43.565079
epoch 840, loss: 43.565071
epoch 841, loss: 43.565067
epoch 842, loss: 43.565067
epoch 843, loss: 43.565063
epoch 844, loss: 43.565060
epoch 845, loss: 43.565056
epoch 846, loss: 43.565052
epoch 847, loss: 43.565048
epoch 848, loss: 43.565048
epoch 849, loss: 43.565044
epoch 850, loss: 43.565041
epoch 851, loss: 43.565037
epoch 852, loss: 43.565033
epoch 853, loss: 43.565033
epoch 854, loss: 43.565029
epoch 855, loss: 43.565025
epoch 856, loss: 43.565022
epoch 857, loss: 43.565018
epoch 858, loss: 43.565014
epoch 859, loss: 43.565014
epoch 860, loss: 43.565010
epoch 861, loss: 43.565006
epoch 862, loss: 43.565002
epoch 863, loss: 43.564999
epoch 864, loss: 43.564999
epoch 865, loss: 43.564995
epoch 866, loss: 43.564991
epoch 867, loss: 43.564987
epoch 868, loss: 43.564983
e

epoch 139, loss: 43.602650
epoch 140, loss: 43.601784
epoch 141, loss: 43.600941
epoch 142, loss: 43.600121
epoch 143, loss: 43.599323
epoch 144, loss: 43.598553
epoch 145, loss: 43.597801
epoch 146, loss: 43.597065
epoch 147, loss: 43.596352
epoch 148, loss: 43.595661
epoch 149, loss: 43.594986
epoch 150, loss: 43.594330
epoch 151, loss: 43.593689
epoch 152, loss: 43.593067
epoch 153, loss: 43.592461
epoch 154, loss: 43.591873
epoch 155, loss: 43.591297
epoch 156, loss: 43.590736
epoch 157, loss: 43.590191
epoch 158, loss: 43.589661
epoch 159, loss: 43.589142
epoch 160, loss: 43.588638
epoch 161, loss: 43.588146
epoch 162, loss: 43.587666
epoch 163, loss: 43.587200
epoch 164, loss: 43.586742
epoch 165, loss: 43.586300
epoch 166, loss: 43.585865
epoch 167, loss: 43.585445
epoch 168, loss: 43.585030
epoch 169, loss: 43.584629
epoch 170, loss: 43.584236
epoch 171, loss: 43.583851
epoch 172, loss: 43.583477
epoch 173, loss: 43.583111
epoch 174, loss: 43.582756
epoch 175, loss: 43.582405
e

epoch 443, loss: 43.564941
epoch 444, loss: 43.564934
epoch 445, loss: 43.564930
epoch 446, loss: 43.564926
epoch 447, loss: 43.564919
epoch 448, loss: 43.564915
epoch 449, loss: 43.564911
epoch 450, loss: 43.564907
epoch 451, loss: 43.564899
epoch 452, loss: 43.564896
epoch 453, loss: 43.564892
epoch 454, loss: 43.564888
epoch 455, loss: 43.564880
epoch 456, loss: 43.564877
epoch 457, loss: 43.564873
epoch 458, loss: 43.564869
epoch 459, loss: 43.564865
epoch 460, loss: 43.564857
epoch 461, loss: 43.564857
epoch 462, loss: 43.564850
epoch 463, loss: 43.564846
epoch 464, loss: 43.564842
epoch 465, loss: 43.564842
epoch 466, loss: 43.564835
epoch 467, loss: 43.564831
epoch 468, loss: 43.564827
epoch 469, loss: 43.564823
epoch 470, loss: 43.564819
epoch 471, loss: 43.564816
epoch 472, loss: 43.564812
epoch 473, loss: 43.564808
epoch 474, loss: 43.564808
epoch 475, loss: 43.564800
epoch 476, loss: 43.564796
epoch 477, loss: 43.564793
epoch 478, loss: 43.564789
epoch 479, loss: 43.564789
e

epoch 747, loss: 43.564503
epoch 748, loss: 43.564503
epoch 749, loss: 43.564503
epoch 750, loss: 43.564503
epoch 751, loss: 43.564503
epoch 752, loss: 43.564503
epoch 753, loss: 43.564503
epoch 754, loss: 43.564503
epoch 755, loss: 43.564503
epoch 756, loss: 43.564503
epoch 757, loss: 43.564499
epoch 758, loss: 43.564499
epoch 759, loss: 43.564499
epoch 760, loss: 43.564499
epoch 761, loss: 43.564499
epoch 762, loss: 43.564499
epoch 763, loss: 43.564499
epoch 764, loss: 43.564499
epoch 765, loss: 43.564499
epoch 766, loss: 43.564499
epoch 767, loss: 43.564499
epoch 768, loss: 43.564499
epoch 769, loss: 43.564499
epoch 770, loss: 43.564499
epoch 771, loss: 43.564499
epoch 772, loss: 43.564499
epoch 773, loss: 43.564499
epoch 774, loss: 43.564499
epoch 775, loss: 43.564499
epoch 776, loss: 43.564499
epoch 777, loss: 43.564499
epoch 778, loss: 43.564499
epoch 779, loss: 43.564495
epoch 780, loss: 43.564495
epoch 781, loss: 43.564495
epoch 782, loss: 43.564495
epoch 783, loss: 43.564495
e

In [30]:
print(rmse)

[9.707947525235276, 9.707942396881121, 9.707875728169423]


In [36]:
%%time
# test RMSE.
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, X_train_all, de_Y_train_all , batch_size=10000, num_epochs= 100, lr = 0.1)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[1] = sqrt(mean_squared_error(de_Y_test ,pred))
print("RMSE = ", rmse_list[1])

epoch 1, loss: 46.264973
epoch 2, loss: 45.762627
epoch 3, loss: 45.687214
epoch 4, loss: 45.661591
epoch 5, loss: 45.667164
epoch 6, loss: 45.652615
epoch 7, loss: 45.657467
epoch 8, loss: 45.660313
epoch 9, loss: 45.655037
epoch 10, loss: 45.642612
epoch 11, loss: 45.654739
epoch 12, loss: 45.638855
epoch 13, loss: 45.641949
epoch 14, loss: 45.652504
epoch 15, loss: 45.647999
epoch 16, loss: 45.656445
epoch 17, loss: 45.664738
epoch 18, loss: 45.655510
epoch 19, loss: 45.647190
epoch 20, loss: 45.644737
epoch 21, loss: 45.644886
epoch 22, loss: 45.643337
epoch 23, loss: 45.652393
epoch 24, loss: 45.648190
epoch 25, loss: 45.650761
epoch 26, loss: 45.658855
epoch 27, loss: 45.655163
epoch 28, loss: 45.650246
epoch 29, loss: 45.650490
epoch 30, loss: 45.660580
epoch 31, loss: 45.644455
epoch 32, loss: 45.655460
epoch 33, loss: 45.645309
epoch 34, loss: 45.649570
epoch 35, loss: 45.657139
epoch 36, loss: 45.651211
epoch 37, loss: 45.648327
epoch 38, loss: 45.653942
epoch 39, loss: 45.64

In [37]:
print("test RMSE = ", rmse_list[1])

test RMSE =  9.510604916798238


#### case 3

In [5]:
# case 3
# MLP_1_dm	de-mean	10,000	MLP, one hidden layer + ReLU	None

In [12]:
%%time
# tune
rmse = [0] * 3
Lr = [0.01, 0.05, 0.1]
for i in range(len(Lr)):
    print(Lr[i])
    net = nn.Sequential()
    net.add(nn.Dense(45, activation='relu'), nn.Dense(1))
    net.initialize(init.Normal(sigma=0.01))
    train(net, features, delabels, batch_size=10000, num_epochs= 1000, lr = Lr[i])
    pred = net(np.array(X_valid)).reshape(1,-1)[0]
    rmse[i] = sqrt(mean_squared_error(de_Y_valid ,pred))
    print("RMSE = ", rmse[i])

0.01
epoch 1, loss: 57.632408
epoch 2, loss: 57.631264
epoch 3, loss: 57.630108
epoch 4, loss: 57.628944
epoch 5, loss: 57.627758
epoch 6, loss: 57.626556
epoch 7, loss: 57.625324
epoch 8, loss: 57.624062
epoch 9, loss: 57.622757
epoch 10, loss: 57.621407
epoch 11, loss: 57.619999
epoch 12, loss: 57.618530
epoch 13, loss: 57.617004
epoch 14, loss: 57.615406
epoch 15, loss: 57.613724
epoch 16, loss: 57.611961
epoch 17, loss: 57.610107
epoch 18, loss: 57.608150
epoch 19, loss: 57.606083
epoch 20, loss: 57.603901
epoch 21, loss: 57.601582
epoch 22, loss: 57.599125
epoch 23, loss: 57.596519
epoch 24, loss: 57.593742
epoch 25, loss: 57.590786
epoch 26, loss: 57.587643
epoch 27, loss: 57.584293
epoch 28, loss: 57.580730
epoch 29, loss: 57.576931
epoch 30, loss: 57.572887
epoch 31, loss: 57.568577
epoch 32, loss: 57.563988
epoch 33, loss: 57.559101
epoch 34, loss: 57.553894
epoch 35, loss: 57.548336
epoch 36, loss: 57.542408
epoch 37, loss: 57.536068
epoch 38, loss: 57.529301
epoch 39, loss: 

epoch 309, loss: 39.425438
epoch 310, loss: 39.416088
epoch 311, loss: 39.406845
epoch 312, loss: 39.397663
epoch 313, loss: 39.388332
epoch 314, loss: 39.379070
epoch 315, loss: 39.369778
epoch 316, loss: 39.360489
epoch 317, loss: 39.351185
epoch 318, loss: 39.342037
epoch 319, loss: 39.332718
epoch 320, loss: 39.323082
epoch 321, loss: 39.313549
epoch 322, loss: 39.304070
epoch 323, loss: 39.294495
epoch 324, loss: 39.284966
epoch 325, loss: 39.275467
epoch 326, loss: 39.265949
epoch 327, loss: 39.256611
epoch 328, loss: 39.247253
epoch 329, loss: 39.238098
epoch 330, loss: 39.228951
epoch 331, loss: 39.219849
epoch 332, loss: 39.210705
epoch 333, loss: 39.201565
epoch 334, loss: 39.192417
epoch 335, loss: 39.183205
epoch 336, loss: 39.174004
epoch 337, loss: 39.164761
epoch 338, loss: 39.155624
epoch 339, loss: 39.146526
epoch 340, loss: 39.137264
epoch 341, loss: 39.128193
epoch 342, loss: 39.118988
epoch 343, loss: 39.109745
epoch 344, loss: 39.100620
epoch 345, loss: 39.091541
e

epoch 614, loss: 37.043827
epoch 615, loss: 37.036949
epoch 616, loss: 37.030010
epoch 617, loss: 37.023212
epoch 618, loss: 37.016308
epoch 619, loss: 37.009224
epoch 620, loss: 37.002384
epoch 621, loss: 36.995449
epoch 622, loss: 36.988640
epoch 623, loss: 36.981705
epoch 624, loss: 36.974884
epoch 625, loss: 36.968006
epoch 626, loss: 36.961021
epoch 627, loss: 36.954250
epoch 628, loss: 36.947243
epoch 629, loss: 36.940441
epoch 630, loss: 36.933578
epoch 631, loss: 36.926594
epoch 632, loss: 36.919697
epoch 633, loss: 36.912827
epoch 634, loss: 36.906181
epoch 635, loss: 36.899185
epoch 636, loss: 36.892181
epoch 637, loss: 36.885342
epoch 638, loss: 36.878372
epoch 639, loss: 36.871441
epoch 640, loss: 36.864586
epoch 641, loss: 36.857761
epoch 642, loss: 36.850735
epoch 643, loss: 36.843781
epoch 644, loss: 36.836914
epoch 645, loss: 36.830002
epoch 646, loss: 36.823059
epoch 647, loss: 36.816143
epoch 648, loss: 36.809330
epoch 649, loss: 36.802361
epoch 650, loss: 36.795265
e

epoch 918, loss: 34.948296
epoch 919, loss: 34.941345
epoch 920, loss: 34.934147
epoch 921, loss: 34.927315
epoch 922, loss: 34.920029
epoch 923, loss: 34.912914
epoch 924, loss: 34.906017
epoch 925, loss: 34.898911
epoch 926, loss: 34.891804
epoch 927, loss: 34.884876
epoch 928, loss: 34.877968
epoch 929, loss: 34.870724
epoch 930, loss: 34.863594
epoch 931, loss: 34.856686
epoch 932, loss: 34.849487
epoch 933, loss: 34.842388
epoch 934, loss: 34.835400
epoch 935, loss: 34.828342
epoch 936, loss: 34.821304
epoch 937, loss: 34.814198
epoch 938, loss: 34.807301
epoch 939, loss: 34.800201
epoch 940, loss: 34.793278
epoch 941, loss: 34.786507
epoch 942, loss: 34.779354
epoch 943, loss: 34.772011
epoch 944, loss: 34.764893
epoch 945, loss: 34.757984
epoch 946, loss: 34.750969
epoch 947, loss: 34.743980
epoch 948, loss: 34.736618
epoch 949, loss: 34.729546
epoch 950, loss: 34.722404
epoch 951, loss: 34.715233
epoch 952, loss: 34.708054
epoch 953, loss: 34.701004
epoch 954, loss: 34.693623
e

epoch 225, loss: 38.265202
epoch 226, loss: 38.492603
epoch 227, loss: 38.623837
epoch 228, loss: 38.563686
epoch 229, loss: 38.344761
epoch 230, loss: 38.040722
epoch 231, loss: 37.676327
epoch 232, loss: 37.333988
epoch 233, loss: 37.045799
epoch 234, loss: 36.820148
epoch 235, loss: 36.652012
epoch 236, loss: 36.519524
epoch 237, loss: 36.437668
epoch 238, loss: 36.372959
epoch 239, loss: 36.362492
epoch 240, loss: 36.360550
epoch 241, loss: 36.426205
epoch 242, loss: 36.505394
epoch 243, loss: 36.688236
epoch 244, loss: 36.889744
epoch 245, loss: 37.192799
epoch 246, loss: 37.524162
epoch 247, loss: 37.946560
epoch 248, loss: 38.292320
epoch 249, loss: 38.555630
epoch 250, loss: 38.585407
epoch 251, loss: 38.422791
epoch 252, loss: 38.076077
epoch 253, loss: 37.613907
epoch 254, loss: 37.196018
epoch 255, loss: 36.814510
epoch 256, loss: 36.544292
epoch 257, loss: 36.322693
epoch 258, loss: 36.167187
epoch 259, loss: 36.051270
epoch 260, loss: 35.985294
epoch 261, loss: 35.950722
e

epoch 528, loss: 33.057423
epoch 529, loss: 32.888264
epoch 530, loss: 32.762325
epoch 531, loss: 32.711239
epoch 532, loss: 32.668602
epoch 533, loss: 32.715221
epoch 534, loss: 32.780697
epoch 535, loss: 32.987900
epoch 536, loss: 33.138821
epoch 537, loss: 33.465958
epoch 538, loss: 33.674213
epoch 539, loss: 34.021168
epoch 540, loss: 34.122978
epoch 541, loss: 34.234592
epoch 542, loss: 34.105137
epoch 543, loss: 33.917999
epoch 544, loss: 33.636555
epoch 545, loss: 33.327484
epoch 546, loss: 33.061089
epoch 547, loss: 32.842426
epoch 548, loss: 32.652298
epoch 549, loss: 32.521126
epoch 550, loss: 32.443581
epoch 551, loss: 32.471092
epoch 552, loss: 32.468338
epoch 553, loss: 32.604939
epoch 554, loss: 32.725964
epoch 555, loss: 33.030659
epoch 556, loss: 33.236458
epoch 557, loss: 33.611675
epoch 558, loss: 33.828331
epoch 559, loss: 34.130749
epoch 560, loss: 34.134876
epoch 561, loss: 34.055218
epoch 562, loss: 33.808273
epoch 563, loss: 33.493992
epoch 564, loss: 33.156811
e

epoch 832, loss: 30.945984
epoch 833, loss: 31.040424
epoch 834, loss: 31.151012
epoch 835, loss: 31.186472
epoch 836, loss: 31.225954
epoch 837, loss: 31.191618
epoch 838, loss: 31.111048
epoch 839, loss: 30.981853
epoch 840, loss: 30.825521
epoch 841, loss: 30.728903
epoch 842, loss: 30.573721
epoch 843, loss: 30.504778
epoch 844, loss: 30.383171
epoch 845, loss: 30.401546
epoch 846, loss: 30.326803
epoch 847, loss: 30.387138
epoch 848, loss: 30.365509
epoch 849, loss: 30.480875
epoch 850, loss: 30.458765
epoch 851, loss: 30.557604
epoch 852, loss: 30.538462
epoch 853, loss: 30.573009
epoch 854, loss: 30.521416
epoch 855, loss: 30.544676
epoch 856, loss: 30.461029
epoch 857, loss: 30.442316
epoch 858, loss: 30.340397
epoch 859, loss: 30.325121
epoch 860, loss: 30.221657
epoch 861, loss: 30.200026
epoch 862, loss: 30.133078
epoch 863, loss: 30.218712
epoch 864, loss: 30.202600
epoch 865, loss: 30.321844
epoch 866, loss: 30.364113
epoch 867, loss: 30.617682
epoch 868, loss: 30.800463
e

epoch 139, loss: 38.626171
epoch 140, loss: 38.599747
epoch 141, loss: 38.812313
epoch 142, loss: 38.571899
epoch 143, loss: 38.566277
epoch 144, loss: 38.167995
epoch 145, loss: 38.080727
epoch 146, loss: 37.792934
epoch 147, loss: 37.766483
epoch 148, loss: 37.667347
epoch 149, loss: 37.759327
epoch 150, loss: 37.810413
epoch 151, loss: 38.015926
epoch 152, loss: 38.129726
epoch 153, loss: 38.325680
epoch 154, loss: 38.300903
epoch 155, loss: 38.309383
epoch 156, loss: 38.034882
epoch 157, loss: 37.954456
epoch 158, loss: 37.676620
epoch 159, loss: 37.729031
epoch 160, loss: 37.651886
epoch 161, loss: 37.971127
epoch 162, loss: 38.026047
epoch 163, loss: 38.588516
epoch 164, loss: 38.508217
epoch 165, loss: 39.302826
epoch 166, loss: 39.783924
epoch 167, loss: 43.211414
epoch 168, loss: 49.502964
epoch 169, loss: 53.107357
epoch 170, loss: 45.634834
epoch 171, loss: 44.934887
epoch 172, loss: 42.252159
epoch 173, loss: 39.419704
epoch 174, loss: 36.988075
epoch 175, loss: 36.455536
e

epoch 444, loss: 51.513020
epoch 445, loss: 39.553623
epoch 446, loss: 37.484085
epoch 447, loss: 38.902565
epoch 448, loss: 39.645622
epoch 449, loss: 33.104816
epoch 450, loss: 31.353928
epoch 451, loss: 30.689522
epoch 452, loss: 30.345423
epoch 453, loss: 30.146019
epoch 454, loss: 30.112375
epoch 455, loss: 30.339212
epoch 456, loss: 31.257454
epoch 457, loss: 33.660088
epoch 458, loss: 39.248768
epoch 459, loss: 44.726982
epoch 460, loss: 44.800362
epoch 461, loss: 36.416473
epoch 462, loss: 32.928326
epoch 463, loss: 31.194319
epoch 464, loss: 30.587322
epoch 465, loss: 30.192619
epoch 466, loss: 30.000479
epoch 467, loss: 29.901726
epoch 468, loss: 29.991806
epoch 469, loss: 30.345232
epoch 470, loss: 31.387703
epoch 471, loss: 33.849514
epoch 472, loss: 38.542427
epoch 473, loss: 43.876369
epoch 474, loss: 41.636555
epoch 475, loss: 36.522305
epoch 476, loss: 36.093864
epoch 477, loss: 36.144672
epoch 478, loss: 36.634106
epoch 479, loss: 32.203125
epoch 480, loss: 31.459326
e

epoch 747, loss: 39.319508
epoch 748, loss: 38.155380
epoch 749, loss: 34.731926
epoch 750, loss: 32.198421
epoch 751, loss: 30.257088
epoch 752, loss: 29.442076
epoch 753, loss: 27.926069
epoch 754, loss: 27.397585
epoch 755, loss: 26.970413
epoch 756, loss: 27.069616
epoch 757, loss: 27.389402
epoch 758, loss: 28.293501
epoch 759, loss: 29.891260
epoch 760, loss: 32.005623
epoch 761, loss: 34.646854
epoch 762, loss: 34.755928
epoch 763, loss: 34.801006
epoch 764, loss: 33.703812
epoch 765, loss: 31.314631
epoch 766, loss: 29.336435
epoch 767, loss: 27.943624
epoch 768, loss: 27.789326
epoch 769, loss: 28.240231
epoch 770, loss: 29.503773
epoch 771, loss: 30.796007
epoch 772, loss: 32.723663
epoch 773, loss: 32.701164
epoch 774, loss: 32.257912
epoch 775, loss: 30.432726
epoch 776, loss: 29.258734
epoch 777, loss: 28.852703
epoch 778, loss: 29.745251
epoch 779, loss: 32.342422
epoch 780, loss: 37.042091
epoch 781, loss: 41.065628
epoch 782, loss: 41.581512
epoch 783, loss: 32.943245
e

In [14]:
print(rmse)

[9.329725647874266, 10.19986284451017, 11.606187125607143]


In [15]:
%%time
#test rmse
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'), nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, features, delabels , batch_size=10000, num_epochs= 1000, lr = 0.01)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[2] = sqrt(mean_squared_error(de_Y_test ,pred))
print("RMSE = ", rmse_list[2])

epoch 1, loss: 57.632313
epoch 2, loss: 57.631031
epoch 3, loss: 57.629738
epoch 4, loss: 57.628437
epoch 5, loss: 57.627113
epoch 6, loss: 57.625771
epoch 7, loss: 57.624393
epoch 8, loss: 57.622982
epoch 9, loss: 57.621532
epoch 10, loss: 57.620029
epoch 11, loss: 57.618481
epoch 12, loss: 57.616867
epoch 13, loss: 57.615189
epoch 14, loss: 57.613438
epoch 15, loss: 57.611607
epoch 16, loss: 57.609688
epoch 17, loss: 57.607670
epoch 18, loss: 57.605545
epoch 19, loss: 57.603313
epoch 20, loss: 57.600948
epoch 21, loss: 57.598450
epoch 22, loss: 57.595806
epoch 23, loss: 57.593014
epoch 24, loss: 57.590057
epoch 25, loss: 57.586926
epoch 26, loss: 57.583607
epoch 27, loss: 57.580082
epoch 28, loss: 57.576336
epoch 29, loss: 57.572361
epoch 30, loss: 57.568138
epoch 31, loss: 57.563644
epoch 32, loss: 57.558861
epoch 33, loss: 57.553764
epoch 34, loss: 57.548351
epoch 35, loss: 57.542576
epoch 36, loss: 57.536430
epoch 37, loss: 57.529888
epoch 38, loss: 57.522911
epoch 39, loss: 57.51

epoch 309, loss: 39.355068
epoch 310, loss: 39.345905
epoch 311, loss: 39.336742
epoch 312, loss: 39.327675
epoch 313, loss: 39.318542
epoch 314, loss: 39.309368
epoch 315, loss: 39.300179
epoch 316, loss: 39.291023
epoch 317, loss: 39.281971
epoch 318, loss: 39.272892
epoch 319, loss: 39.263905
epoch 320, loss: 39.255058
epoch 321, loss: 39.246216
epoch 322, loss: 39.237358
epoch 323, loss: 39.228394
epoch 324, loss: 39.219421
epoch 325, loss: 39.210636
epoch 326, loss: 39.201885
epoch 327, loss: 39.193039
epoch 328, loss: 39.184200
epoch 329, loss: 39.175430
epoch 330, loss: 39.166668
epoch 331, loss: 39.157997
epoch 332, loss: 39.149364
epoch 333, loss: 39.140770
epoch 334, loss: 39.132320
epoch 335, loss: 39.123920
epoch 336, loss: 39.115543
epoch 337, loss: 39.107227
epoch 338, loss: 39.098980
epoch 339, loss: 39.090763
epoch 340, loss: 39.082645
epoch 341, loss: 39.074543
epoch 342, loss: 39.066349
epoch 343, loss: 39.058052
epoch 344, loss: 39.049892
epoch 345, loss: 39.041748
e

epoch 613, loss: 37.102493
epoch 614, loss: 37.095795
epoch 615, loss: 37.089092
epoch 616, loss: 37.082527
epoch 617, loss: 37.075825
epoch 618, loss: 37.069164
epoch 619, loss: 37.062450
epoch 620, loss: 37.055820
epoch 621, loss: 37.049198
epoch 622, loss: 37.042564
epoch 623, loss: 37.035900
epoch 624, loss: 37.029285
epoch 625, loss: 37.022846
epoch 626, loss: 37.016247
epoch 627, loss: 37.009689
epoch 628, loss: 37.003204
epoch 629, loss: 36.996651
epoch 630, loss: 36.990211
epoch 631, loss: 36.983631
epoch 632, loss: 36.977261
epoch 633, loss: 36.970703
epoch 634, loss: 36.964439
epoch 635, loss: 36.957886
epoch 636, loss: 36.951527
epoch 637, loss: 36.945099
epoch 638, loss: 36.938698
epoch 639, loss: 36.932297
epoch 640, loss: 36.925930
epoch 641, loss: 36.919441
epoch 642, loss: 36.912914
epoch 643, loss: 36.906517
epoch 644, loss: 36.900208
epoch 645, loss: 36.893764
epoch 646, loss: 36.887478
epoch 647, loss: 36.880947
epoch 648, loss: 36.874645
epoch 649, loss: 36.868103
e

epoch 918, loss: 35.115459
epoch 919, loss: 35.108486
epoch 920, loss: 35.101604
epoch 921, loss: 35.094559
epoch 922, loss: 35.087627
epoch 923, loss: 35.080479
epoch 924, loss: 35.073441
epoch 925, loss: 35.066460
epoch 926, loss: 35.059345
epoch 927, loss: 35.052380
epoch 928, loss: 35.045544
epoch 929, loss: 35.038502
epoch 930, loss: 35.031555
epoch 931, loss: 35.024746
epoch 932, loss: 35.017925
epoch 933, loss: 35.010841
epoch 934, loss: 35.003910
epoch 935, loss: 34.996792
epoch 936, loss: 34.989670
epoch 937, loss: 34.982594
epoch 938, loss: 34.975674
epoch 939, loss: 34.968800
epoch 940, loss: 34.961983
epoch 941, loss: 34.954895
epoch 942, loss: 34.947838
epoch 943, loss: 34.940758
epoch 944, loss: 34.933559
epoch 945, loss: 34.926731
epoch 946, loss: 34.919559
epoch 947, loss: 34.912155
epoch 948, loss: 34.904533
epoch 949, loss: 34.897381
epoch 950, loss: 34.889744
epoch 951, loss: 34.882431
epoch 952, loss: 34.875076
epoch 953, loss: 34.867821
epoch 954, loss: 34.860317
e

In [16]:
print("test RMSE = ", rmse_list[2])

test RMSE =  9.30458862643002


#### case 4

In [ ]:
# case 4
# MLP_2_dm	de-mean	10,000	MLP, two hidden layers + ReLu	None

In [22]:
%%time
# tune 
rmse = [0] * 3
Lr = [0.01, 0.05, 0.1]
for i in range(len(Lr)):
    print(Lr[i])
    net = nn.Sequential()
    net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
    net.initialize(init.Normal(sigma=0.01))
    train(net, features, delabels, batch_size=10000, num_epochs= 1000, lr = Lr[i])
    pred = net(np.array(X_valid)).reshape(1,-1)[0]
    rmse[i] = sqrt(mean_squared_error(de_Y_valid ,pred))
    print("RMSE = ", rmse[i])

0.01
epoch 1, loss: 57.634205
epoch 2, loss: 57.634144
epoch 3, loss: 57.634075
epoch 4, loss: 57.634007
epoch 5, loss: 57.633945
epoch 6, loss: 57.633881
epoch 7, loss: 57.633820
epoch 8, loss: 57.633755
epoch 9, loss: 57.633701
epoch 10, loss: 57.633636
epoch 11, loss: 57.633583
epoch 12, loss: 57.633526
epoch 13, loss: 57.633469
epoch 14, loss: 57.633411
epoch 15, loss: 57.633362
epoch 16, loss: 57.633305
epoch 17, loss: 57.633255
epoch 18, loss: 57.633205
epoch 19, loss: 57.633156
epoch 20, loss: 57.633106
epoch 21, loss: 57.633064
epoch 22, loss: 57.633011
epoch 23, loss: 57.632969
epoch 24, loss: 57.632919
epoch 25, loss: 57.632874
epoch 26, loss: 57.632832
epoch 27, loss: 57.632786
epoch 28, loss: 57.632751
epoch 29, loss: 57.632706
epoch 30, loss: 57.632668
epoch 31, loss: 57.632626
epoch 32, loss: 57.632587
epoch 33, loss: 57.632549
epoch 34, loss: 57.632511
epoch 35, loss: 57.632477
epoch 36, loss: 57.632439
epoch 37, loss: 57.632401
epoch 38, loss: 57.632362
epoch 39, loss: 

epoch 309, loss: 57.626682
epoch 310, loss: 57.626633
epoch 311, loss: 57.626583
epoch 312, loss: 57.626530
epoch 313, loss: 57.626476
epoch 314, loss: 57.626427
epoch 315, loss: 57.626369
epoch 316, loss: 57.626312
epoch 317, loss: 57.626255
epoch 318, loss: 57.626202
epoch 319, loss: 57.626144
epoch 320, loss: 57.626080
epoch 321, loss: 57.626026
epoch 322, loss: 57.625961
epoch 323, loss: 57.625900
epoch 324, loss: 57.625839
epoch 325, loss: 57.625774
epoch 326, loss: 57.625713
epoch 327, loss: 57.625648
epoch 328, loss: 57.625580
epoch 329, loss: 57.625511
epoch 330, loss: 57.625450
epoch 331, loss: 57.625374
epoch 332, loss: 57.625305
epoch 333, loss: 57.625237
epoch 334, loss: 57.625164
epoch 335, loss: 57.625095
epoch 336, loss: 57.625019
epoch 337, loss: 57.624943
epoch 338, loss: 57.624863
epoch 339, loss: 57.624786
epoch 340, loss: 57.624706
epoch 341, loss: 57.624626
epoch 342, loss: 57.624542
epoch 343, loss: 57.624462
epoch 344, loss: 57.624374
epoch 345, loss: 57.624294
e

epoch 613, loss: 43.040455
epoch 614, loss: 42.858044
epoch 615, loss: 42.684387
epoch 616, loss: 42.519176
epoch 617, loss: 42.360352
epoch 618, loss: 42.208744
epoch 619, loss: 42.064152
epoch 620, loss: 41.926285
epoch 621, loss: 41.793552
epoch 622, loss: 41.666908
epoch 623, loss: 41.545181
epoch 624, loss: 41.429699
epoch 625, loss: 41.319412
epoch 626, loss: 41.213924
epoch 627, loss: 41.113548
epoch 628, loss: 41.018417
epoch 629, loss: 40.928024
epoch 630, loss: 40.842537
epoch 631, loss: 40.760124
epoch 632, loss: 40.681568
epoch 633, loss: 40.605751
epoch 634, loss: 40.533188
epoch 635, loss: 40.463455
epoch 636, loss: 40.396580
epoch 637, loss: 40.332542
epoch 638, loss: 40.271152
epoch 639, loss: 40.211460
epoch 640, loss: 40.153458
epoch 641, loss: 40.097336
epoch 642, loss: 40.042908
epoch 643, loss: 39.989822
epoch 644, loss: 39.938408
epoch 645, loss: 39.888451
epoch 646, loss: 39.839836
epoch 647, loss: 39.792770
epoch 648, loss: 39.746986
epoch 649, loss: 39.702587
e

epoch 917, loss: 33.023781
epoch 918, loss: 32.695080
epoch 919, loss: 32.789467
epoch 920, loss: 32.554977
epoch 921, loss: 32.698769
epoch 922, loss: 32.538586
epoch 923, loss: 32.790901
epoch 924, loss: 32.682728
epoch 925, loss: 33.073002
epoch 926, loss: 33.008499
epoch 927, loss: 33.602184
epoch 928, loss: 33.391438
epoch 929, loss: 33.835705
epoch 930, loss: 33.269367
epoch 931, loss: 33.299610
epoch 932, loss: 32.724445
epoch 933, loss: 32.636452
epoch 934, loss: 32.261467
epoch 935, loss: 32.281223
epoch 936, loss: 32.066303
epoch 937, loss: 32.191822
epoch 938, loss: 32.085304
epoch 939, loss: 32.345898
epoch 940, loss: 32.305546
epoch 941, loss: 32.780094
epoch 942, loss: 32.831158
epoch 943, loss: 33.514126
epoch 944, loss: 33.291233
epoch 945, loss: 33.634003
epoch 946, loss: 32.974499
epoch 947, loss: 32.838314
epoch 948, loss: 32.226852
epoch 949, loss: 32.079857
epoch 950, loss: 31.768665
epoch 951, loss: 31.782230
epoch 952, loss: 31.630968
epoch 953, loss: 31.787096
e

epoch 224, loss: 40.060883
epoch 225, loss: 42.498791
epoch 226, loss: 38.063110
epoch 227, loss: 35.320778
epoch 228, loss: 34.814186
epoch 229, loss: 34.597282
epoch 230, loss: 34.549358
epoch 231, loss: 34.713387
epoch 232, loss: 35.417526
epoch 233, loss: 36.772434
epoch 234, loss: 40.122208
epoch 235, loss: 38.245842
epoch 236, loss: 35.801601
epoch 237, loss: 34.965267
epoch 238, loss: 34.525974
epoch 239, loss: 35.123695
epoch 240, loss: 34.898071
epoch 241, loss: 36.396538
epoch 242, loss: 35.398556
epoch 243, loss: 36.655788
epoch 244, loss: 34.758698
epoch 245, loss: 35.225372
epoch 246, loss: 35.282017
epoch 247, loss: 39.545467
epoch 248, loss: 37.283775
epoch 249, loss: 39.269039
epoch 250, loss: 34.490345
epoch 251, loss: 34.112541
epoch 252, loss: 34.095444
epoch 253, loss: 35.612354
epoch 254, loss: 34.570530
epoch 255, loss: 35.949429
epoch 256, loss: 34.472874
epoch 257, loss: 36.432789
epoch 258, loss: 35.588383
epoch 259, loss: 39.282257
epoch 260, loss: 33.810143
e

epoch 528, loss: 26.523106
epoch 529, loss: 21.507677
epoch 530, loss: 25.092445
epoch 531, loss: 20.600365
epoch 532, loss: 22.784668
epoch 533, loss: 20.433836
epoch 534, loss: 23.534229
epoch 535, loss: 21.222235
epoch 536, loss: 26.374725
epoch 537, loss: 21.291401
epoch 538, loss: 24.848484
epoch 539, loss: 20.436333
epoch 540, loss: 23.015415
epoch 541, loss: 19.909388
epoch 542, loss: 21.921253
epoch 543, loss: 20.282309
epoch 544, loss: 24.570032
epoch 545, loss: 21.222836
epoch 546, loss: 26.357538
epoch 547, loss: 20.450033
epoch 548, loss: 22.927328
epoch 549, loss: 19.958794
epoch 550, loss: 22.569830
epoch 551, loss: 20.066187
epoch 552, loss: 23.700071
epoch 553, loss: 20.639980
epoch 554, loss: 25.182209
epoch 555, loss: 20.475639
epoch 556, loss: 24.022058
epoch 557, loss: 19.972193
epoch 558, loss: 22.607832
epoch 559, loss: 19.895451
epoch 560, loss: 23.207432
epoch 561, loss: 20.116665
epoch 562, loss: 23.719591
epoch 563, loss: 20.144258
epoch 564, loss: 23.871174
e

epoch 832, loss: 16.951176
epoch 833, loss: 15.489441
epoch 834, loss: 17.716694
epoch 835, loss: 16.046707
epoch 836, loss: 19.429466
epoch 837, loss: 16.587576
epoch 838, loss: 20.484470
epoch 839, loss: 16.285921
epoch 840, loss: 18.850067
epoch 841, loss: 15.641961
epoch 842, loss: 17.413574
epoch 843, loss: 15.413319
epoch 844, loss: 17.199696
epoch 845, loss: 15.382656
epoch 846, loss: 17.497108
epoch 847, loss: 15.529157
epoch 848, loss: 18.046690
epoch 849, loss: 15.663178
epoch 850, loss: 18.203941
epoch 851, loss: 15.816759
epoch 852, loss: 18.875284
epoch 853, loss: 16.475092
epoch 854, loss: 20.360720
epoch 855, loss: 15.594621
epoch 856, loss: 17.434856
epoch 857, loss: 15.587301
epoch 858, loss: 18.680809
epoch 859, loss: 15.448089
epoch 860, loss: 17.208492
epoch 861, loss: 15.032894
epoch 862, loss: 16.656303
epoch 863, loss: 15.169988
epoch 864, loss: 17.340687
epoch 865, loss: 15.885650
epoch 866, loss: 19.566725
epoch 867, loss: 16.301775
epoch 868, loss: 19.893150
e

epoch 139, loss: 38.156788
epoch 140, loss: 39.129978
epoch 141, loss: 38.986485
epoch 142, loss: 42.867126
epoch 143, loss: 37.491116
epoch 144, loss: 37.036053
epoch 145, loss: 37.364697
epoch 146, loss: 39.001549
epoch 147, loss: 47.979996
epoch 148, loss: 48.798325
epoch 149, loss: 47.288189
epoch 150, loss: 48.221191
epoch 151, loss: 40.651997
epoch 152, loss: 37.478237
epoch 153, loss: 38.054863
epoch 154, loss: 39.737453
epoch 155, loss: 47.943096
epoch 156, loss: 44.109505
epoch 157, loss: 38.919498
epoch 158, loss: 36.015209
epoch 159, loss: 35.685661
epoch 160, loss: 35.864960
epoch 161, loss: 37.682755
epoch 162, loss: 44.478012
epoch 163, loss: 45.782578
epoch 164, loss: 45.659969
epoch 165, loss: 44.680450
epoch 166, loss: 48.843163
epoch 167, loss: 42.890858
epoch 168, loss: 39.790852
epoch 169, loss: 40.238277
epoch 170, loss: 45.817230
epoch 171, loss: 42.308083
epoch 172, loss: 41.568443
epoch 173, loss: 40.133167
epoch 174, loss: 40.095135
epoch 175, loss: 36.544449
e

epoch 443, loss: 22.489019
epoch 444, loss: 22.683485
epoch 445, loss: 28.717985
epoch 446, loss: 23.906658
epoch 447, loss: 24.748323
epoch 448, loss: 20.199722
epoch 449, loss: 20.239920
epoch 450, loss: 19.241972
epoch 451, loss: 19.903843
epoch 452, loss: 19.949192
epoch 453, loss: 21.701761
epoch 454, loss: 21.661642
epoch 455, loss: 25.398695
epoch 456, loss: 21.510115
epoch 457, loss: 23.819056
epoch 458, loss: 20.701746
epoch 459, loss: 21.724009
epoch 460, loss: 19.803535
epoch 461, loss: 20.087751
epoch 462, loss: 19.127821
epoch 463, loss: 20.650501
epoch 464, loss: 20.427963
epoch 465, loss: 22.922787
epoch 466, loss: 21.021507
epoch 467, loss: 23.744473
epoch 468, loss: 21.669098
epoch 469, loss: 27.165604
epoch 470, loss: 22.804192
epoch 471, loss: 23.072083
epoch 472, loss: 18.894344
epoch 473, loss: 18.899342
epoch 474, loss: 17.999031
epoch 475, loss: 18.261292
epoch 476, loss: 18.228830
epoch 477, loss: 19.453590
epoch 478, loss: 19.713820
epoch 479, loss: 21.946756
e

epoch 746, loss: 16.648233
epoch 747, loss: 20.195091
epoch 748, loss: 16.338154
epoch 749, loss: 18.003189
epoch 750, loss: 14.743892
epoch 751, loss: 15.272975
epoch 752, loss: 13.984375
epoch 753, loss: 14.553554
epoch 754, loss: 13.881172
epoch 755, loss: 14.855393
epoch 756, loss: 13.765363
epoch 757, loss: 14.457241
epoch 758, loss: 13.679954
epoch 759, loss: 14.725446
epoch 760, loss: 14.275858
epoch 761, loss: 16.284737
epoch 762, loss: 15.520253
epoch 763, loss: 19.038824
epoch 764, loss: 16.342907
epoch 765, loss: 19.096441
epoch 766, loss: 15.022309
epoch 767, loss: 15.859334
epoch 768, loss: 13.932495
epoch 769, loss: 14.513895
epoch 770, loss: 13.690423
epoch 771, loss: 14.469986
epoch 772, loss: 13.391952
epoch 773, loss: 14.035539
epoch 774, loss: 13.324003
epoch 775, loss: 14.114008
epoch 776, loss: 13.710230
epoch 777, loss: 15.157043
epoch 778, loss: 14.837037
epoch 779, loss: 17.960402
epoch 780, loss: 16.273678
epoch 781, loss: 20.565512
epoch 782, loss: 16.286655
e

In [23]:
print(rmse)

[9.385465468150993, 10.266153907295585, 11.144458435973064]


In [24]:
%%time
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, features, delabels , batch_size=10000, num_epochs= 1000, lr = 0.01)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[3] = sqrt(mean_squared_error(de_Y_test ,pred))
print("RMSE = ", rmse_list[3])

epoch 1, loss: 57.634094
epoch 2, loss: 57.634026
epoch 3, loss: 57.633961
epoch 4, loss: 57.633892
epoch 5, loss: 57.633831
epoch 6, loss: 57.633770
epoch 7, loss: 57.633705
epoch 8, loss: 57.633644
epoch 9, loss: 57.633587
epoch 10, loss: 57.633526
epoch 11, loss: 57.633469
epoch 12, loss: 57.633411
epoch 13, loss: 57.633358
epoch 14, loss: 57.633305
epoch 15, loss: 57.633251
epoch 16, loss: 57.633202
epoch 17, loss: 57.633148
epoch 18, loss: 57.633099
epoch 19, loss: 57.633049
epoch 20, loss: 57.632999
epoch 21, loss: 57.632950
epoch 22, loss: 57.632908
epoch 23, loss: 57.632862
epoch 24, loss: 57.632812
epoch 25, loss: 57.632771
epoch 26, loss: 57.632725
epoch 27, loss: 57.632687
epoch 28, loss: 57.632645
epoch 29, loss: 57.632599
epoch 30, loss: 57.632561
epoch 31, loss: 57.632526
epoch 32, loss: 57.632481
epoch 33, loss: 57.632442
epoch 34, loss: 57.632408
epoch 35, loss: 57.632370
epoch 36, loss: 57.632336
epoch 37, loss: 57.632301
epoch 38, loss: 57.632263
epoch 39, loss: 57.63

epoch 309, loss: 57.625648
epoch 310, loss: 57.625580
epoch 311, loss: 57.625519
epoch 312, loss: 57.625450
epoch 313, loss: 57.625374
epoch 314, loss: 57.625305
epoch 315, loss: 57.625233
epoch 316, loss: 57.625164
epoch 317, loss: 57.625088
epoch 318, loss: 57.625011
epoch 319, loss: 57.624931
epoch 320, loss: 57.624855
epoch 321, loss: 57.624775
epoch 322, loss: 57.624695
epoch 323, loss: 57.624611
epoch 324, loss: 57.624531
epoch 325, loss: 57.624443
epoch 326, loss: 57.624355
epoch 327, loss: 57.624268
epoch 328, loss: 57.624180
epoch 329, loss: 57.624092
epoch 330, loss: 57.624001
epoch 331, loss: 57.623905
epoch 332, loss: 57.623814
epoch 333, loss: 57.623711
epoch 334, loss: 57.623619
epoch 335, loss: 57.623520
epoch 336, loss: 57.623413
epoch 337, loss: 57.623314
epoch 338, loss: 57.623207
epoch 339, loss: 57.623100
epoch 340, loss: 57.622986
epoch 341, loss: 57.622883
epoch 342, loss: 57.622768
epoch 343, loss: 57.622650
epoch 344, loss: 57.622536
epoch 345, loss: 57.622417
e

epoch 613, loss: 40.737427
epoch 614, loss: 40.655125
epoch 615, loss: 40.575977
epoch 616, loss: 40.499916
epoch 617, loss: 40.428730
epoch 618, loss: 40.361759
epoch 619, loss: 40.297623
epoch 620, loss: 40.236370
epoch 621, loss: 40.177864
epoch 622, loss: 40.121338
epoch 623, loss: 40.066929
epoch 624, loss: 40.014149
epoch 625, loss: 39.963097
epoch 626, loss: 39.913239
epoch 627, loss: 39.865143
epoch 628, loss: 39.819672
epoch 629, loss: 39.775528
epoch 630, loss: 39.732662
epoch 631, loss: 39.691204
epoch 632, loss: 39.650936
epoch 633, loss: 39.611629
epoch 634, loss: 39.573097
epoch 635, loss: 39.535240
epoch 636, loss: 39.497696
epoch 637, loss: 39.460609
epoch 638, loss: 39.424576
epoch 639, loss: 39.389374
epoch 640, loss: 39.354492
epoch 641, loss: 39.320236
epoch 642, loss: 39.286972
epoch 643, loss: 39.254112
epoch 644, loss: 39.221741
epoch 645, loss: 39.189861
epoch 646, loss: 39.157913
epoch 647, loss: 39.126045
epoch 648, loss: 39.094627
epoch 649, loss: 39.063251
e

epoch 917, loss: 32.591942
epoch 918, loss: 32.494236
epoch 919, loss: 32.565559
epoch 920, loss: 32.558102
epoch 921, loss: 32.796864
epoch 922, loss: 32.891129
epoch 923, loss: 33.352409
epoch 924, loss: 33.429028
epoch 925, loss: 33.962341
epoch 926, loss: 33.739456
epoch 927, loss: 33.868114
epoch 928, loss: 33.314156
epoch 929, loss: 33.141205
epoch 930, loss: 32.655102
epoch 931, loss: 32.480106
epoch 932, loss: 32.259415
epoch 933, loss: 32.209911
epoch 934, loss: 32.114758
epoch 935, loss: 32.175064
epoch 936, loss: 32.168324
epoch 937, loss: 32.389633
epoch 938, loss: 32.520981
epoch 939, loss: 33.023514
epoch 940, loss: 33.136608
epoch 941, loss: 33.679779
epoch 942, loss: 33.490944
epoch 943, loss: 33.671200
epoch 944, loss: 33.095055
epoch 945, loss: 32.844753
epoch 946, loss: 32.328426
epoch 947, loss: 32.108437
epoch 948, loss: 31.820610
epoch 949, loss: 31.732925
epoch 950, loss: 31.604706
epoch 951, loss: 31.648182
epoch 952, loss: 31.631403
epoch 953, loss: 31.809929
e

In [41]:
print("test RMSE = ", rmse_list[3])

test RMSE =  9.347290462945796


#### case 5

In [ ]:
# case 5(weight decay)
# MLP_2_dm_L2	de-mean	10,000	MLP, two hidden layers + ReLu	Weight Decay (L2)

In [33]:
rmse = [[0,0,0], [0,0,0]]

In [36]:
%%time
# tune
rmse = [[0,0,0], [0,0,0]]

Lr = [0.05, 0.1]
Wd = [0.1, 0.5, 1]
for i in range(len(Lr)):
    for j in range(len(Wd)):
        print(Lr[i], Wd[j])
        net = nn.Sequential()
        net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
        net.initialize(init.Normal(sigma=0.01))
        train(net, features, delabels, batch_size=10000, num_epochs= 1000, lr = Lr[i], wd = Wd[j])
        pred = net(np.array(X_valid)).reshape(1,-1)[0]
        rmse[i][j] = sqrt(mean_squared_error(de_Y_valid ,pred))
        print("RMSE = ", rmse[i][j])

0.05 0.1
epoch 1, loss: 57.633900
epoch 2, loss: 57.633606
epoch 3, loss: 57.633343
epoch 4, loss: 57.633106
epoch 5, loss: 57.632893
epoch 6, loss: 57.632702
epoch 7, loss: 57.632530
epoch 8, loss: 57.632370
epoch 9, loss: 57.632225
epoch 10, loss: 57.632099
epoch 11, loss: 57.631981
epoch 12, loss: 57.631874
epoch 13, loss: 57.631783
epoch 14, loss: 57.631695
epoch 15, loss: 57.631611
epoch 16, loss: 57.631538
epoch 17, loss: 57.631474
epoch 18, loss: 57.631413
epoch 19, loss: 57.631363
epoch 20, loss: 57.631313
epoch 21, loss: 57.631264
epoch 22, loss: 57.631226
epoch 23, loss: 57.631180
epoch 24, loss: 57.631149
epoch 25, loss: 57.631111
epoch 26, loss: 57.631088
epoch 27, loss: 57.631058
epoch 28, loss: 57.631031
epoch 29, loss: 57.631008
epoch 30, loss: 57.630981
epoch 31, loss: 57.630962
epoch 32, loss: 57.630936
epoch 33, loss: 57.630920
epoch 34, loss: 57.630901
epoch 35, loss: 57.630882
epoch 36, loss: 57.630863
epoch 37, loss: 57.630852
epoch 38, loss: 57.630833
epoch 39, lo

epoch 309, loss: 40.076820
epoch 310, loss: 46.272514
epoch 311, loss: 42.578541
epoch 312, loss: 37.532803
epoch 313, loss: 36.269165
epoch 314, loss: 35.945091
epoch 315, loss: 35.894329
epoch 316, loss: 36.048561
epoch 317, loss: 37.101173
epoch 318, loss: 37.587837
epoch 319, loss: 40.996685
epoch 320, loss: 37.759354
epoch 321, loss: 36.833862
epoch 322, loss: 35.956470
epoch 323, loss: 36.269230
epoch 324, loss: 36.914017
epoch 325, loss: 40.000553
epoch 326, loss: 41.907986
epoch 327, loss: 38.958614
epoch 328, loss: 36.046394
epoch 329, loss: 35.338619
epoch 330, loss: 35.206520
epoch 331, loss: 35.354492
epoch 332, loss: 35.916164
epoch 333, loss: 37.833035
epoch 334, loss: 40.668709
epoch 335, loss: 40.318562
epoch 336, loss: 37.093067
epoch 337, loss: 35.496830
epoch 338, loss: 35.746262
epoch 339, loss: 36.282467
epoch 340, loss: 39.203472
epoch 341, loss: 38.102787
epoch 342, loss: 41.562477
epoch 343, loss: 38.160599
epoch 344, loss: 37.907455
epoch 345, loss: 35.865891
e

epoch 613, loss: 32.717052
epoch 614, loss: 31.466244
epoch 615, loss: 30.463732
epoch 616, loss: 29.683468
epoch 617, loss: 29.723949
epoch 618, loss: 32.801186
epoch 619, loss: 31.263866
epoch 620, loss: 34.042404
epoch 621, loss: 29.040874
epoch 622, loss: 27.860020
epoch 623, loss: 27.579878
epoch 624, loss: 28.661316
epoch 625, loss: 30.159359
epoch 626, loss: 36.976273
epoch 627, loss: 29.752913
epoch 628, loss: 27.276510
epoch 629, loss: 26.869925
epoch 630, loss: 27.164856
epoch 631, loss: 30.051373
epoch 632, loss: 34.501652
epoch 633, loss: 38.768440
epoch 634, loss: 37.884655
epoch 635, loss: 31.907532
epoch 636, loss: 28.776007
epoch 637, loss: 27.262541
epoch 638, loss: 27.118584
epoch 639, loss: 27.376492
epoch 640, loss: 30.619669
epoch 641, loss: 31.269228
epoch 642, loss: 40.241489
epoch 643, loss: 29.287287
epoch 644, loss: 27.569065
epoch 645, loss: 26.554934
epoch 646, loss: 27.500628
epoch 647, loss: 29.791584
epoch 648, loss: 38.956055
epoch 649, loss: 42.466507
e

epoch 917, loss: 21.495380
epoch 918, loss: 22.903288
epoch 919, loss: 35.421318
epoch 920, loss: 23.333870
epoch 921, loss: 21.262102
epoch 922, loss: 21.271030
epoch 923, loss: 24.313820
epoch 924, loss: 43.600613
epoch 925, loss: 164.817764
epoch 926, loss: 508.319092
epoch 927, loss: 57.639580
epoch 928, loss: 57.600475
epoch 929, loss: 57.478306
epoch 930, loss: 57.090481
epoch 931, loss: 56.822994
epoch 932, loss: 56.720570
epoch 933, loss: 56.510468
epoch 934, loss: 57.255013
epoch 935, loss: 56.967937
epoch 936, loss: 55.477989
epoch 937, loss: 54.597599
epoch 938, loss: 53.439175
epoch 939, loss: 54.083157
epoch 940, loss: 199.286438
epoch 941, loss: 58.266369
epoch 942, loss: 58.122192
epoch 943, loss: 58.068943
epoch 944, loss: 58.021355
epoch 945, loss: 57.978825
epoch 946, loss: 57.940811
epoch 947, loss: 57.906837
epoch 948, loss: 57.876461
epoch 949, loss: 57.849319
epoch 950, loss: 57.825058
epoch 951, loss: 57.803375
epoch 952, loss: 57.783974
epoch 953, loss: 57.76664

epoch 9, loss: 57.631264
epoch 10, loss: 57.631149
epoch 11, loss: 57.631058
epoch 12, loss: 57.630974
epoch 13, loss: 57.630901
epoch 14, loss: 57.630836
epoch 15, loss: 57.630783
epoch 16, loss: 57.630730
epoch 17, loss: 57.630680
epoch 18, loss: 57.630638
epoch 19, loss: 57.630592
epoch 20, loss: 57.630550
epoch 21, loss: 57.630512
epoch 22, loss: 57.630474
epoch 23, loss: 57.630436
epoch 24, loss: 57.630394
epoch 25, loss: 57.630356
epoch 26, loss: 57.630318
epoch 27, loss: 57.630276
epoch 28, loss: 57.630238
epoch 29, loss: 57.630196
epoch 30, loss: 57.630150
epoch 31, loss: 57.630100
epoch 32, loss: 57.630051
epoch 33, loss: 57.630001
epoch 34, loss: 57.629951
epoch 35, loss: 57.629894
epoch 36, loss: 57.629829
epoch 37, loss: 57.629768
epoch 38, loss: 57.629707
epoch 39, loss: 57.629631
epoch 40, loss: 57.629555
epoch 41, loss: 57.629482
epoch 42, loss: 57.629395
epoch 43, loss: 57.629299
epoch 44, loss: 57.629200
epoch 45, loss: 57.629093
epoch 46, loss: 57.628983
epoch 47, los

epoch 316, loss: 41.824898
epoch 317, loss: 36.605682
epoch 318, loss: 39.521160
epoch 319, loss: 34.922119
epoch 320, loss: 36.301231
epoch 321, loss: 37.019913
epoch 322, loss: 44.716145
epoch 323, loss: 37.361763
epoch 324, loss: 41.535389
epoch 325, loss: 36.403057
epoch 326, loss: 34.842621
epoch 327, loss: 37.987846
epoch 328, loss: 36.349976
epoch 329, loss: 41.205605
epoch 330, loss: 36.371979
epoch 331, loss: 40.948425
epoch 332, loss: 36.597111
epoch 333, loss: 43.566109
epoch 334, loss: 38.408272
epoch 335, loss: 34.582592
epoch 336, loss: 34.786945
epoch 337, loss: 35.197544
epoch 338, loss: 41.110222
epoch 339, loss: 37.547310
epoch 340, loss: 43.650620
epoch 341, loss: 36.776680
epoch 342, loss: 38.535580
epoch 343, loss: 33.486137
epoch 344, loss: 32.819210
epoch 345, loss: 33.104633
epoch 346, loss: 37.470711
epoch 347, loss: 39.813084
epoch 348, loss: 51.806324
epoch 349, loss: 38.326420
epoch 350, loss: 33.543949
epoch 351, loss: 33.010914
epoch 352, loss: 32.847809
e

epoch 620, loss: 32.256794
epoch 621, loss: 31.648869
epoch 622, loss: 30.725225
epoch 623, loss: 29.498587
epoch 624, loss: 34.729130
epoch 625, loss: 33.685841
epoch 626, loss: 41.541817
epoch 627, loss: 31.975512
epoch 628, loss: 30.327620
epoch 629, loss: 28.628069
epoch 630, loss: 31.491693
epoch 631, loss: 34.233212
epoch 632, loss: 45.996578
epoch 633, loss: 32.944721
epoch 634, loss: 29.051407
epoch 635, loss: 27.508213
epoch 636, loss: 27.635622
epoch 637, loss: 29.455772
epoch 638, loss: 39.642166
epoch 639, loss: 41.329834
epoch 640, loss: 36.139252
epoch 641, loss: 33.514267
epoch 642, loss: 34.626396
epoch 643, loss: 46.740143
epoch 644, loss: 41.802193
epoch 645, loss: 38.657749
epoch 646, loss: 36.218277
epoch 647, loss: 34.748363
epoch 648, loss: 35.435883
epoch 649, loss: 38.846745
epoch 650, loss: 36.285900
epoch 651, loss: 41.320896
epoch 652, loss: 34.640575
epoch 653, loss: 34.295429
epoch 654, loss: 31.768209
epoch 655, loss: 32.389908
epoch 656, loss: 32.537506
e

epoch 924, loss: 31.068432
epoch 925, loss: 32.481785
epoch 926, loss: 29.159275
epoch 927, loss: 32.275391
epoch 928, loss: 30.946629
epoch 929, loss: 36.295925
epoch 930, loss: 30.204931
epoch 931, loss: 30.496447
epoch 932, loss: 30.497215
epoch 933, loss: 34.082314
epoch 934, loss: 28.987791
epoch 935, loss: 27.930565
epoch 936, loss: 27.825804
epoch 937, loss: 31.071497
epoch 938, loss: 32.219429
epoch 939, loss: 32.913223
epoch 940, loss: 27.321898
epoch 941, loss: 26.254206
epoch 942, loss: 26.225906
epoch 943, loss: 28.012138
epoch 944, loss: 30.632822
epoch 945, loss: 40.981449
epoch 946, loss: 32.975452
epoch 947, loss: 43.982323
epoch 948, loss: 34.128769
epoch 949, loss: 31.168854
epoch 950, loss: 31.648140
epoch 951, loss: 33.402008
epoch 952, loss: 44.067024
epoch 953, loss: 39.438385
epoch 954, loss: 41.485676
epoch 955, loss: 37.269657
epoch 956, loss: 38.836330
epoch 957, loss: 33.524387
epoch 958, loss: 30.962175
epoch 959, loss: 30.701988
epoch 960, loss: 32.584167
e

In [37]:
print(rmse)

[[11.026708892157863, 11.033153154554551, 11.033053984026148], [10.300310286931557, 11.033153233100556, 11.033053991863003]]


In [38]:
%%time
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, features, delabels, batch_size=10000, num_epochs= 1000, lr =0.1 , wd =0.1)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[4] = sqrt(mean_squared_error(de_Y_test ,pred))
print("RMSE = ", rmse_list[4])

epoch 1, loss: 57.633606
epoch 2, loss: 57.633076
epoch 3, loss: 57.632645
epoch 4, loss: 57.632294
epoch 5, loss: 57.632008
epoch 6, loss: 57.631767
epoch 7, loss: 57.631577
epoch 8, loss: 57.631413
epoch 9, loss: 57.631275
epoch 10, loss: 57.631157
epoch 11, loss: 57.631050
epoch 12, loss: 57.630962
epoch 13, loss: 57.630882
epoch 14, loss: 57.630806
epoch 15, loss: 57.630737
epoch 16, loss: 57.630676
epoch 17, loss: 57.630611
epoch 18, loss: 57.630558
epoch 19, loss: 57.630505
epoch 20, loss: 57.630451
epoch 21, loss: 57.630402
epoch 22, loss: 57.630348
epoch 23, loss: 57.630299
epoch 24, loss: 57.630249
epoch 25, loss: 57.630199
epoch 26, loss: 57.630150
epoch 27, loss: 57.630100
epoch 28, loss: 57.630043
epoch 29, loss: 57.629986
epoch 30, loss: 57.629925
epoch 31, loss: 57.629868
epoch 32, loss: 57.629799
epoch 33, loss: 57.629730
epoch 34, loss: 57.629662
epoch 35, loss: 57.629581
epoch 36, loss: 57.629501
epoch 37, loss: 57.629414
epoch 38, loss: 57.629318
epoch 39, loss: 57.62

epoch 309, loss: 45.163357
epoch 310, loss: 40.264023
epoch 311, loss: 36.161842
epoch 312, loss: 34.643723
epoch 313, loss: 36.762009
epoch 314, loss: 33.394352
epoch 315, loss: 35.482880
epoch 316, loss: 32.762527
epoch 317, loss: 35.382713
epoch 318, loss: 34.777454
epoch 319, loss: 37.705120
epoch 320, loss: 34.078320
epoch 321, loss: 34.315552
epoch 322, loss: 32.027386
epoch 323, loss: 37.011211
epoch 324, loss: 33.435459
epoch 325, loss: 32.910351
epoch 326, loss: 32.073738
epoch 327, loss: 37.834743
epoch 328, loss: 34.237087
epoch 329, loss: 35.456367
epoch 330, loss: 30.742109
epoch 331, loss: 33.077393
epoch 332, loss: 32.377041
epoch 333, loss: 39.349186
epoch 334, loss: 33.465595
epoch 335, loss: 34.136288
epoch 336, loss: 31.295048
epoch 337, loss: 31.198853
epoch 338, loss: 32.680836
epoch 339, loss: 44.523514
epoch 340, loss: 37.917797
epoch 341, loss: 38.250217
epoch 342, loss: 33.327061
epoch 343, loss: 29.460340
epoch 344, loss: 29.185698
epoch 345, loss: 31.749250
e

epoch 613, loss: 30.315451
epoch 614, loss: 33.585030
epoch 615, loss: 30.923798
epoch 616, loss: 34.721481
epoch 617, loss: 30.122175
epoch 618, loss: 30.887976
epoch 619, loss: 29.091578
epoch 620, loss: 30.643885
epoch 621, loss: 29.580744
epoch 622, loss: 33.439129
epoch 623, loss: 32.317123
epoch 624, loss: 41.904549
epoch 625, loss: 30.052298
epoch 626, loss: 32.151848
epoch 627, loss: 29.644806
epoch 628, loss: 35.079739
epoch 629, loss: 29.172649
epoch 630, loss: 33.016907
epoch 631, loss: 30.676979
epoch 632, loss: 38.080917
epoch 633, loss: 30.376600
epoch 634, loss: 34.486042
epoch 635, loss: 30.252659
epoch 636, loss: 33.330624
epoch 637, loss: 29.833841
epoch 638, loss: 34.142693
epoch 639, loss: 29.857843
epoch 640, loss: 34.784721
epoch 641, loss: 29.578199
epoch 642, loss: 33.878983
epoch 643, loss: 29.640831
epoch 644, loss: 34.306892
epoch 645, loss: 31.341209
epoch 646, loss: 38.120869
epoch 647, loss: 30.298780
epoch 648, loss: 34.091881
epoch 649, loss: 30.002003
e

epoch 916, loss: 29.305876
epoch 917, loss: 34.574306
epoch 918, loss: 28.247454
epoch 919, loss: 32.519077
epoch 920, loss: 29.023926
epoch 921, loss: 35.450188
epoch 922, loss: 28.336981
epoch 923, loss: 31.619978
epoch 924, loss: 28.804216
epoch 925, loss: 34.997898
epoch 926, loss: 28.094816
epoch 927, loss: 31.678709
epoch 928, loss: 27.633381
epoch 929, loss: 33.381840
epoch 930, loss: 27.724924
epoch 931, loss: 31.408237
epoch 932, loss: 27.707779
epoch 933, loss: 33.831955
epoch 934, loss: 28.559134
epoch 935, loss: 34.769600
epoch 936, loss: 28.878349
epoch 937, loss: 34.273090
epoch 938, loss: 29.595537
epoch 939, loss: 36.312660
epoch 940, loss: 28.558607
epoch 941, loss: 31.141159
epoch 942, loss: 28.427763
epoch 943, loss: 34.107880
epoch 944, loss: 27.669334
epoch 945, loss: 31.586349
epoch 946, loss: 27.648937
epoch 947, loss: 32.366085
epoch 948, loss: 27.510910
epoch 949, loss: 32.399933
epoch 950, loss: 27.944366
epoch 951, loss: 34.765205
epoch 952, loss: 30.797613
e

In [40]:
print("test RMSE = ", rmse_list[4])

test RMSE =  9.899236047314359


#### case 6

In [ ]:
# case 6(dropout)
# MLP_2_dm_dropout	de-mean	10,000	MLP, two hidden layers + ReLu	Dropout

In [42]:
%%time
# tune
rmse = [[0,0,0], [0,0,0]]

Lr = [0.05, 0.1]
Drop = [0.2, 0.5, 0.8]
for i in range(len(Lr)):
    for j in range(len(Drop)):
        print(Lr[i], Drop[j])
        net = nn.Sequential()
        net.add(nn.Dense(45, activation='relu'),
                 nn.Dropout(Drop[j]),
                 nn.Dense(45, activation='relu'),
                 nn.Dropout(Drop[j]),
                 nn.Dense(1))
        net.initialize(init.Normal(sigma=0.01))
        train(net, features, delabels, batch_size=10000, num_epochs= 1000, lr = Lr[i])
        pred = net(np.array(X_valid)).reshape(1,-1)[0]
        rmse[i][j] = sqrt(mean_squared_error(de_Y_valid ,pred))
        print("RMSE = ", rmse[i][j])

0.05 0.2
epoch 1, loss: 57.633812
epoch 2, loss: 57.633526
epoch 3, loss: 57.633263
epoch 4, loss: 57.633026
epoch 5, loss: 57.632805
epoch 6, loss: 57.632607
epoch 7, loss: 57.632423
epoch 8, loss: 57.632263
epoch 9, loss: 57.632111
epoch 10, loss: 57.631973
epoch 11, loss: 57.631844
epoch 12, loss: 57.631733
epoch 13, loss: 57.631626
epoch 14, loss: 57.631523
epoch 15, loss: 57.631432
epoch 16, loss: 57.631344
epoch 17, loss: 57.631268
epoch 18, loss: 57.631195
epoch 19, loss: 57.631126
epoch 20, loss: 57.631058
epoch 21, loss: 57.630993
epoch 22, loss: 57.630936
epoch 23, loss: 57.630882
epoch 24, loss: 57.630825
epoch 25, loss: 57.630775
epoch 26, loss: 57.630726
epoch 27, loss: 57.630680
epoch 28, loss: 57.630630
epoch 29, loss: 57.630589
epoch 30, loss: 57.630539
epoch 31, loss: 57.630493
epoch 32, loss: 57.630455
epoch 33, loss: 57.630413
epoch 34, loss: 57.630367
epoch 35, loss: 57.630318
epoch 36, loss: 57.630276
epoch 37, loss: 57.630230
epoch 38, loss: 57.630188
epoch 39, lo

epoch 309, loss: 31.000635
epoch 310, loss: 31.600393
epoch 311, loss: 30.059912
epoch 312, loss: 32.544140
epoch 313, loss: 30.292997
epoch 314, loss: 33.179436
epoch 315, loss: 30.311960
epoch 316, loss: 32.247585
epoch 317, loss: 29.847744
epoch 318, loss: 31.338276
epoch 319, loss: 29.890022
epoch 320, loss: 33.178669
epoch 321, loss: 30.119526
epoch 322, loss: 31.713537
epoch 323, loss: 29.429546
epoch 324, loss: 31.091616
epoch 325, loss: 31.265743
epoch 326, loss: 33.786594
epoch 327, loss: 38.762756
epoch 328, loss: 38.034538
epoch 329, loss: 31.570646
epoch 330, loss: 29.834831
epoch 331, loss: 29.073681
epoch 332, loss: 28.951885
epoch 333, loss: 28.401209
epoch 334, loss: 28.292372
epoch 335, loss: 28.014372
epoch 336, loss: 28.236650
epoch 337, loss: 28.169252
epoch 338, loss: 30.136963
epoch 339, loss: 30.265703
epoch 340, loss: 31.602297
epoch 341, loss: 33.292412
epoch 342, loss: 30.195265
epoch 343, loss: 29.299479
epoch 344, loss: 28.080622
epoch 345, loss: 28.171656
e

epoch 613, loss: 22.651691
epoch 614, loss: 19.503603
epoch 615, loss: 23.703936
epoch 616, loss: 19.850540
epoch 617, loss: 23.707865
epoch 618, loss: 19.811462
epoch 619, loss: 23.526072
epoch 620, loss: 19.737225
epoch 621, loss: 23.983307
epoch 622, loss: 19.775005
epoch 623, loss: 23.329300
epoch 624, loss: 19.394842
epoch 625, loss: 21.057512
epoch 626, loss: 19.001440
epoch 627, loss: 20.808065
epoch 628, loss: 18.904123
epoch 629, loss: 21.532793
epoch 630, loss: 19.021170
epoch 631, loss: 22.480614
epoch 632, loss: 19.351353
epoch 633, loss: 22.138184
epoch 634, loss: 19.180302
epoch 635, loss: 21.208902
epoch 636, loss: 18.910990
epoch 637, loss: 21.520767
epoch 638, loss: 19.288153
epoch 639, loss: 23.770590
epoch 640, loss: 19.406202
epoch 641, loss: 23.863855
epoch 642, loss: 19.324863
epoch 643, loss: 21.667969
epoch 644, loss: 18.969852
epoch 645, loss: 21.752966
epoch 646, loss: 18.852089
epoch 647, loss: 21.483713
epoch 648, loss: 18.962229
epoch 649, loss: 22.652632
e

epoch 917, loss: 19.545626
epoch 918, loss: 16.661554
epoch 919, loss: 18.080740
epoch 920, loss: 16.362663
epoch 921, loss: 17.828194
epoch 922, loss: 16.324507
epoch 923, loss: 18.124697
epoch 924, loss: 16.325323
epoch 925, loss: 19.355198
epoch 926, loss: 16.561726
epoch 927, loss: 20.006062
epoch 928, loss: 16.764715
epoch 929, loss: 19.250549
epoch 930, loss: 16.525528
epoch 931, loss: 17.894985
epoch 932, loss: 16.186811
epoch 933, loss: 16.873388
epoch 934, loss: 16.024637
epoch 935, loss: 16.965059
epoch 936, loss: 16.035826
epoch 937, loss: 17.075075
epoch 938, loss: 15.982155
epoch 939, loss: 17.802376
epoch 940, loss: 16.041578
epoch 941, loss: 18.777470
epoch 942, loss: 16.395542
epoch 943, loss: 18.562613
epoch 944, loss: 16.434757
epoch 945, loss: 17.866053
epoch 946, loss: 16.338507
epoch 947, loss: 18.859058
epoch 948, loss: 16.567921
epoch 949, loss: 18.536638
epoch 950, loss: 16.354166
epoch 951, loss: 18.050064
epoch 952, loss: 16.237709
epoch 953, loss: 18.979599
e

epoch 224, loss: 35.857143
epoch 225, loss: 35.897152
epoch 226, loss: 35.874218
epoch 227, loss: 35.541920
epoch 228, loss: 35.372208
epoch 229, loss: 35.027245
epoch 230, loss: 35.187954
epoch 231, loss: 35.174519
epoch 232, loss: 35.132034
epoch 233, loss: 35.362015
epoch 234, loss: 35.199047
epoch 235, loss: 35.274597
epoch 236, loss: 35.177715
epoch 237, loss: 34.954002
epoch 238, loss: 35.503674
epoch 239, loss: 34.970261
epoch 240, loss: 35.270855
epoch 241, loss: 34.769402
epoch 242, loss: 34.902889
epoch 243, loss: 34.984436
epoch 244, loss: 35.414478
epoch 245, loss: 34.958168
epoch 246, loss: 34.941528
epoch 247, loss: 34.443344
epoch 248, loss: 34.680714
epoch 249, loss: 34.478680
epoch 250, loss: 34.740993
epoch 251, loss: 34.053505
epoch 252, loss: 34.433617
epoch 253, loss: 34.213093
epoch 254, loss: 35.033318
epoch 255, loss: 34.731983
epoch 256, loss: 34.942966
epoch 257, loss: 34.049458
epoch 258, loss: 34.050339
epoch 259, loss: 33.454777
epoch 260, loss: 33.860363
e

...

epoch 577, loss: 24.099943
epoch 578, loss: 23.169321
epoch 579, loss: 23.600910
epoch 580, loss: 23.165028
epoch 581, loss: 23.517712
epoch 582, loss: 23.289116
epoch 583, loss: 24.512892
epoch 584, loss: 22.676847
epoch 585, loss: 24.217201
epoch 586, loss: 23.210766
epoch 587, loss: 23.549383
epoch 588, loss: 23.355444
epoch 589, loss: 23.669004
epoch 590, loss: 24.024305
epoch 591, loss: 24.186872
epoch 592, loss: 23.886566
epoch 593, loss: 23.797201
epoch 594, loss: 23.426943
epoch 595, loss: 23.523872
epoch 596, loss: 23.522783
epoch 597, loss: 23.839108
epoch 598, loss: 23.097275
epoch 599, loss: 24.443037
epoch 600, loss: 22.820227
epoch 601, loss: 24.223808
epoch 602, loss: 22.700214
epoch 603, loss: 25.562281
epoch 604, loss: 23.235928
epoch 605, loss: 25.074463
epoch 606, loss: 22.760677
epoch 607, loss: 25.418129
epoch 608, loss: 22.798323
epoch 609, loss: 25.659723
epoch 610, loss: 22.948750
epoch 611, loss: 27.202724
epoch 612, loss: 22.963346
epoch 613, loss: 27.699984
e

epoch 881, loss: 22.484022
epoch 882, loss: 21.898355
epoch 883, loss: 22.285213
epoch 884, loss: 22.080400
epoch 885, loss: 22.423214
epoch 886, loss: 21.438078
epoch 887, loss: 23.222088
epoch 888, loss: 21.678841
epoch 889, loss: 23.887457
epoch 890, loss: 22.816771
epoch 891, loss: 24.927290
epoch 892, loss: 21.882265
epoch 893, loss: 24.692966
epoch 894, loss: 22.525600
epoch 895, loss: 24.102306
epoch 896, loss: 21.899399
epoch 897, loss: 23.258089
epoch 898, loss: 21.773964
epoch 899, loss: 23.058485
epoch 900, loss: 21.847418
epoch 901, loss: 24.925518
epoch 902, loss: 22.190218
epoch 903, loss: 22.978170
epoch 904, loss: 21.903219
epoch 905, loss: 22.762196
epoch 906, loss: 22.078382
epoch 907, loss: 22.421402
epoch 908, loss: 22.141434
epoch 909, loss: 22.732197
epoch 910, loss: 21.960470
epoch 911, loss: 23.063406
epoch 912, loss: 21.656784
epoch 913, loss: 23.854290
epoch 914, loss: 21.846563
epoch 915, loss: 24.161638
epoch 916, loss: 21.915985
epoch 917, loss: 23.205181
e

epoch 188, loss: 41.516594
epoch 189, loss: 40.749908
epoch 190, loss: 40.022270
epoch 191, loss: 40.278198
epoch 192, loss: 41.346260
epoch 193, loss: 39.372517
epoch 194, loss: 41.110580
epoch 195, loss: 39.016430
epoch 196, loss: 40.576797
epoch 197, loss: 39.377617
epoch 198, loss: 41.393314
epoch 199, loss: 39.352852
epoch 200, loss: 39.907368
epoch 201, loss: 39.200180
epoch 202, loss: 40.261242
epoch 203, loss: 39.952011
epoch 204, loss: 40.905689
epoch 205, loss: 39.477932
epoch 206, loss: 40.598450
epoch 207, loss: 40.141655
epoch 208, loss: 40.713764
epoch 209, loss: 39.740829
epoch 210, loss: 40.773716
epoch 211, loss: 38.933086
epoch 212, loss: 40.231091
epoch 213, loss: 40.334324
epoch 214, loss: 40.679642
epoch 215, loss: 39.169384
epoch 216, loss: 39.903580
epoch 217, loss: 39.325024
epoch 218, loss: 39.606030
epoch 219, loss: 39.350777
epoch 220, loss: 39.666317
epoch 221, loss: 40.285065
epoch 222, loss: 41.154324
epoch 223, loss: 39.569122
epoch 224, loss: 39.987740
e

epoch 492, loss: 39.730484
epoch 493, loss: 37.076721
epoch 494, loss: 38.840565
epoch 495, loss: 37.255272
epoch 496, loss: 39.392677
epoch 497, loss: 36.731987
epoch 498, loss: 39.088432
epoch 499, loss: 37.067211
epoch 500, loss: 39.431530
epoch 501, loss: 36.395775
epoch 502, loss: 39.825748
epoch 503, loss: 37.204594
epoch 504, loss: 37.797050
epoch 505, loss: 37.730530
epoch 506, loss: 39.347076
epoch 507, loss: 38.211380
epoch 508, loss: 38.002995
epoch 509, loss: 36.914921
epoch 510, loss: 39.174942
epoch 511, loss: 37.306175
epoch 512, loss: 39.117840
epoch 513, loss: 36.596535
epoch 514, loss: 39.717911
epoch 515, loss: 36.615700
epoch 516, loss: 38.772514
epoch 517, loss: 36.714195
epoch 518, loss: 39.312752
epoch 519, loss: 36.892902
epoch 520, loss: 39.103249
epoch 521, loss: 37.539692
epoch 522, loss: 39.745636
epoch 523, loss: 38.998035
epoch 524, loss: 37.253242
epoch 525, loss: 37.863117
epoch 526, loss: 39.253319
epoch 527, loss: 37.666824
epoch 528, loss: 38.796101
e

epoch 796, loss: 37.985268
epoch 797, loss: 36.382366
epoch 798, loss: 40.128525
epoch 799, loss: 36.938900
epoch 800, loss: 35.585087
epoch 801, loss: 37.526436
epoch 802, loss: 36.051632
epoch 803, loss: 36.891636
epoch 804, loss: 38.271439
epoch 805, loss: 36.570610
epoch 806, loss: 39.412716
epoch 807, loss: 36.710930
epoch 808, loss: 38.283676
epoch 809, loss: 36.073299
epoch 810, loss: 39.980114
epoch 811, loss: 36.747547
epoch 812, loss: 37.829880
epoch 813, loss: 35.708710
epoch 814, loss: 40.598293
epoch 815, loss: 37.845745
epoch 816, loss: 36.766937
epoch 817, loss: 36.202858
epoch 818, loss: 39.524551
epoch 819, loss: 36.329891
epoch 820, loss: 38.192547
epoch 821, loss: 37.027466
epoch 822, loss: 39.818382
epoch 823, loss: 37.367058
epoch 824, loss: 36.489052
epoch 825, loss: 36.095158
epoch 826, loss: 38.191952
epoch 827, loss: 36.133656
epoch 828, loss: 39.549606
epoch 829, loss: 36.755398
epoch 830, loss: 38.199280
epoch 831, loss: 36.608185
epoch 832, loss: 39.399723
e

In [43]:
print(rmse)

[[9.742715657370505, 9.571950510490787, 9.453934458030435], [9.974865764193037, 9.670043145451432, 9.490290255574745]]


In [44]:
%%time
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'),
         nn.Dropout(0.8),
         nn.Dense(45, activation='relu'),
         nn.Dropout(0.8),
         nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, features, delabels, batch_size=10000, num_epochs= 1000, lr = 0.05)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[5] = sqrt(mean_squared_error(de_Y_test ,pred))
print("test RMSE = ", rmse_list[5])

epoch 1, loss: 57.633606
epoch 2, loss: 57.633305
epoch 3, loss: 57.633018
epoch 4, loss: 57.632763
epoch 5, loss: 57.632530
epoch 6, loss: 57.632320
epoch 7, loss: 57.632111
epoch 8, loss: 57.631939
epoch 9, loss: 57.631775
epoch 10, loss: 57.631630
epoch 11, loss: 57.631489
epoch 12, loss: 57.631355
epoch 13, loss: 57.631237
epoch 14, loss: 57.631119
epoch 15, loss: 57.631008
epoch 16, loss: 57.630905
epoch 17, loss: 57.630806
epoch 18, loss: 57.630711
epoch 19, loss: 57.630627
epoch 20, loss: 57.630531
epoch 21, loss: 57.630444
epoch 22, loss: 57.630363
epoch 23, loss: 57.630276
epoch 24, loss: 57.630196
epoch 25, loss: 57.630100
epoch 26, loss: 57.630020
epoch 27, loss: 57.629932
epoch 28, loss: 57.629856
epoch 29, loss: 57.629780
epoch 30, loss: 57.629700
epoch 31, loss: 57.629601
epoch 32, loss: 57.629513
epoch 33, loss: 57.629417
epoch 34, loss: 57.629318
epoch 35, loss: 57.629230
epoch 36, loss: 57.629120
epoch 37, loss: 57.629013
epoch 38, loss: 57.628899
epoch 39, loss: 57.62

epoch 309, loss: 38.281395
epoch 310, loss: 38.184494
epoch 311, loss: 38.152546
epoch 312, loss: 38.126022
epoch 313, loss: 38.223751
epoch 314, loss: 37.736404
epoch 315, loss: 38.008205
epoch 316, loss: 38.003151
epoch 317, loss: 38.013836
epoch 318, loss: 37.957317
epoch 319, loss: 38.381706
epoch 320, loss: 37.876972
epoch 321, loss: 38.118698
epoch 322, loss: 37.826614
epoch 323, loss: 38.189312
epoch 324, loss: 37.659542
epoch 325, loss: 37.810204
epoch 326, loss: 37.829823
epoch 327, loss: 37.857426
epoch 328, loss: 38.112801
epoch 329, loss: 37.841740
epoch 330, loss: 38.158005
epoch 331, loss: 37.594688
epoch 332, loss: 38.004593
epoch 333, loss: 37.888664
epoch 334, loss: 38.072823
epoch 335, loss: 37.834618
epoch 336, loss: 38.221851
epoch 337, loss: 37.575882
epoch 338, loss: 38.146832
epoch 339, loss: 37.442986
epoch 340, loss: 38.049881
epoch 341, loss: 37.759945
epoch 342, loss: 37.873791
epoch 343, loss: 37.694679
epoch 344, loss: 37.683517
epoch 345, loss: 37.907253
e

epoch 613, loss: 35.926544
epoch 614, loss: 35.990780
epoch 615, loss: 36.400883
epoch 616, loss: 35.817841
epoch 617, loss: 36.184597
epoch 618, loss: 35.945366
epoch 619, loss: 35.854568
epoch 620, loss: 35.840012
epoch 621, loss: 35.636967
epoch 622, loss: 35.801380
epoch 623, loss: 35.693764
epoch 624, loss: 35.955742
epoch 625, loss: 35.988605
epoch 626, loss: 35.872368
epoch 627, loss: 35.738049
epoch 628, loss: 35.927631
epoch 629, loss: 36.111946
epoch 630, loss: 35.733788
epoch 631, loss: 35.325832
epoch 632, loss: 36.424698
epoch 633, loss: 35.235439
epoch 634, loss: 36.508446
epoch 635, loss: 35.262970
epoch 636, loss: 36.420761
epoch 637, loss: 35.463539
epoch 638, loss: 36.166039
epoch 639, loss: 35.518276
epoch 640, loss: 36.062241
epoch 641, loss: 35.864880
epoch 642, loss: 36.004128
epoch 643, loss: 35.490849
epoch 644, loss: 35.963490
epoch 645, loss: 35.168633
epoch 646, loss: 36.431721
epoch 647, loss: 35.319874
epoch 648, loss: 36.456551
epoch 649, loss: 35.518967
e

epoch 917, loss: 34.237061
epoch 918, loss: 36.173607
epoch 919, loss: 34.452427
epoch 920, loss: 34.983631
epoch 921, loss: 35.262196
epoch 922, loss: 35.512905
epoch 923, loss: 34.464897
epoch 924, loss: 35.914516
epoch 925, loss: 34.531021
epoch 926, loss: 35.488144
epoch 927, loss: 34.766216
epoch 928, loss: 35.132950
epoch 929, loss: 34.480434
epoch 930, loss: 35.215477
epoch 931, loss: 35.109787
epoch 932, loss: 34.906094
epoch 933, loss: 34.419834
epoch 934, loss: 35.554859
epoch 935, loss: 34.176559
epoch 936, loss: 35.821121
epoch 937, loss: 34.360184
epoch 938, loss: 35.343994
epoch 939, loss: 35.041508
epoch 940, loss: 35.715122
epoch 941, loss: 35.021580
epoch 942, loss: 35.328568
epoch 943, loss: 34.893726
epoch 944, loss: 35.508389
epoch 945, loss: 34.858391
epoch 946, loss: 34.589039
epoch 947, loss: 35.557251
epoch 948, loss: 35.660126
epoch 949, loss: 34.974083
epoch 950, loss: 35.882629
epoch 951, loss: 35.025482
epoch 952, loss: 35.606586
epoch 953, loss: 34.908798
e

In [45]:
print("test RMSE = ", rmse_list[5])

test RMSE =  9.36024760864741


#### case 7

In [ ]:
# case 7
# MLP_2_ykeep	keep	10,000	MLP, two hidden layers + ReLu	None
# non-de-mean diff form case 4

In [64]:
%%time
# tune
rmse = [0] * 2
Lr = [0.001, 0.0005]
for i in range(len(Lr)):
    print(Lr[i])
    net = nn.Sequential()
    net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
    net.initialize(init.Normal(sigma=0.01))
    train(net, features, labels, batch_size=10000, num_epochs= 1000, lr = Lr[i])
    
    pred = net(np.array(X_valid)).reshape(1,-1)[0]
    rmse[i] = sqrt(mean_squared_error(Y_valid ,pred))
    print("RMSE = ", rmse[i])

0.001
epoch 1, loss: 1992993.625000
epoch 2, loss: 1988979.500000
epoch 3, loss: 1984723.125000
epoch 4, loss: 1978111.125000
epoch 5, loss: 1944192.625000
epoch 6, loss: 389984.875000
epoch 7, loss: 43816030208.000000
epoch 8, loss: 1083150889385984.000000
epoch 9, loss: 429290272.000000
epoch 10, loss: 428432128.000000
epoch 11, loss: 427575712.000000
epoch 12, loss: 426721024.000000
epoch 13, loss: 425868032.000000
epoch 14, loss: 425016736.000000
epoch 15, loss: 424167104.000000
epoch 16, loss: 423319168.000000
epoch 17, loss: 422472960.000000
epoch 18, loss: 421628384.000000
epoch 19, loss: 420785568.000000
epoch 20, loss: 419944416.000000
epoch 21, loss: 419104960.000000
epoch 22, loss: 418267200.000000
epoch 23, loss: 417431040.000000
epoch 24, loss: 416596608.000000
epoch 25, loss: 415763808.000000
epoch 26, loss: 414932704.000000
epoch 27, loss: 414103264.000000
epoch 28, loss: 413275456.000000
epoch 29, loss: 412449312.000000
epoch 30, loss: 411624864.000000
epoch 31, loss: 4

epoch 246, loss: 267173072.000000
epoch 247, loss: 266639008.000000
epoch 248, loss: 266105968.000000
epoch 249, loss: 265574000.000000
epoch 250, loss: 265043136.000000
epoch 251, loss: 264513312.000000
epoch 252, loss: 263984560.000000
epoch 253, loss: 263456816.000000
epoch 254, loss: 262930176.000000
epoch 255, loss: 262404576.000000
epoch 256, loss: 261880048.000000
epoch 257, loss: 261356512.000000
epoch 258, loss: 260834064.000000
epoch 259, loss: 260312656.000000
epoch 260, loss: 259792304.000000
epoch 261, loss: 259272976.000000
epoch 262, loss: 258754688.000000
epoch 263, loss: 258237456.000000
epoch 264, loss: 257721232.000000
epoch 265, loss: 257206048.000000
epoch 266, loss: 256691904.000000
epoch 267, loss: 256178784.000000
epoch 268, loss: 255666688.000000
epoch 269, loss: 255155600.000000
epoch 270, loss: 254645552.000000
epoch 271, loss: 254136496.000000
epoch 272, loss: 253628464.000000
epoch 273, loss: 253121504.000000
epoch 274, loss: 252615504.000000
epoch 275, los

epoch 487, loss: 164952192.000000
epoch 488, loss: 164622448.000000
epoch 489, loss: 164293376.000000
epoch 490, loss: 163964944.000000
epoch 491, loss: 163637184.000000
epoch 492, loss: 163310080.000000
epoch 493, loss: 162983632.000000
epoch 494, loss: 162657824.000000
epoch 495, loss: 162332656.000000
epoch 496, loss: 162008176.000000
epoch 497, loss: 161684320.000000
epoch 498, loss: 161361120.000000
epoch 499, loss: 161038560.000000
epoch 500, loss: 160716624.000000
epoch 501, loss: 160395360.000000
epoch 502, loss: 160074752.000000
epoch 503, loss: 159754768.000000
epoch 504, loss: 159435408.000000
epoch 505, loss: 159116688.000000
epoch 506, loss: 158798608.000000
epoch 507, loss: 158481168.000000
epoch 508, loss: 158164384.000000
epoch 509, loss: 157848224.000000
epoch 510, loss: 157532688.000000
epoch 511, loss: 157217776.000000
epoch 512, loss: 156903504.000000
epoch 513, loss: 156589840.000000
epoch 514, loss: 156276816.000000
epoch 515, loss: 155964416.000000
epoch 516, los

epoch 728, loss: 101840992.000000
epoch 729, loss: 101637408.000000
epoch 730, loss: 101434232.000000
epoch 731, loss: 101231464.000000
epoch 732, loss: 101029112.000000
epoch 733, loss: 100827152.000000
epoch 734, loss: 100625608.000000
epoch 735, loss: 100424448.000000
epoch 736, loss: 100223704.000000
epoch 737, loss: 100023360.000000
epoch 738, loss: 99823416.000000
epoch 739, loss: 99623864.000000
epoch 740, loss: 99424720.000000
epoch 741, loss: 99225968.000000
epoch 742, loss: 99027616.000000
epoch 743, loss: 98829664.000000
epoch 744, loss: 98632096.000000
epoch 745, loss: 98434936.000000
epoch 746, loss: 98238152.000000
epoch 747, loss: 98041784.000000
epoch 748, loss: 97845800.000000
epoch 749, loss: 97650200.000000
epoch 750, loss: 97455008.000000
epoch 751, loss: 97260192.000000
epoch 752, loss: 97065768.000000
epoch 753, loss: 96871720.000000
epoch 754, loss: 96678088.000000
epoch 755, loss: 96484824.000000
epoch 756, loss: 96291944.000000
epoch 757, loss: 96099456.000000


epoch 976, loss: 62001880.000000
epoch 977, loss: 61877936.000000
epoch 978, loss: 61754240.000000
epoch 979, loss: 61630788.000000
epoch 980, loss: 61507588.000000
epoch 981, loss: 61384640.000000
epoch 982, loss: 61261932.000000
epoch 983, loss: 61139464.000000
epoch 984, loss: 61017252.000000
epoch 985, loss: 60895276.000000
epoch 986, loss: 60773544.000000
epoch 987, loss: 60652068.000000
epoch 988, loss: 60530824.000000
epoch 989, loss: 60409820.000000
epoch 990, loss: 60289056.000000
epoch 991, loss: 60168544.000000
epoch 992, loss: 60048272.000000
epoch 993, loss: 59928228.000000
epoch 994, loss: 59808428.000000
epoch 995, loss: 59688880.000000
epoch 996, loss: 59569564.000000
epoch 997, loss: 59450484.000000
epoch 998, loss: 59331640.000000
epoch 999, loss: 59213036.000000
epoch 1000, loss: 59094664.000000
RMSE =  10871.607766346315
0.0005
epoch 1, loss: 1994989.750000
epoch 2, loss: 1992986.375000
epoch 3, loss: 1990957.500000
epoch 4, loss: 1988817.875000
epoch 5, loss: 19862

epoch 234, loss: 2437566.750000
epoch 235, loss: 2435129.750000
epoch 236, loss: 2432695.750000
epoch 237, loss: 2430263.250000
epoch 238, loss: 2427833.750000
epoch 239, loss: 2425406.500000
epoch 240, loss: 2422982.000000
epoch 241, loss: 2420559.750000
epoch 242, loss: 2418140.000000
epoch 243, loss: 2415722.500000
epoch 244, loss: 2413307.000000
epoch 245, loss: 2410894.750000
epoch 246, loss: 2408484.500000
epoch 247, loss: 2406076.500000
epoch 248, loss: 2403670.750000
epoch 249, loss: 2401268.000000
epoch 250, loss: 2398867.500000
epoch 251, loss: 2396469.250000
epoch 252, loss: 2394073.750000
epoch 253, loss: 2391680.000000
epoch 254, loss: 2389289.000000
epoch 255, loss: 2386900.250000
epoch 256, loss: 2384514.250000
epoch 257, loss: 2382130.250000
epoch 258, loss: 2379748.750000
epoch 259, loss: 2377369.500000
epoch 260, loss: 2374992.750000
epoch 261, loss: 2372618.750000
epoch 262, loss: 2370246.750000
epoch 263, loss: 2367877.000000
epoch 264, loss: 2365509.750000
epoch 26

epoch 491, loss: 1885028.500000
epoch 492, loss: 1883144.000000
epoch 493, loss: 1881261.500000
epoch 494, loss: 1879381.000000
epoch 495, loss: 1877502.000000
epoch 496, loss: 1875625.125000
epoch 497, loss: 1873749.750000
epoch 498, loss: 1871876.500000
epoch 499, loss: 1870005.250000
epoch 500, loss: 1868135.875000
epoch 501, loss: 1866268.250000
epoch 502, loss: 1864402.500000
epoch 503, loss: 1862538.500000
epoch 504, loss: 1860676.625000
epoch 505, loss: 1858816.250000
epoch 506, loss: 1856958.000000
epoch 507, loss: 1855101.750000
epoch 508, loss: 1853247.125000
epoch 509, loss: 1851394.250000
epoch 510, loss: 1849543.500000
epoch 511, loss: 1847694.500000
epoch 512, loss: 1845847.250000
epoch 513, loss: 1844001.750000
epoch 514, loss: 1842158.625000
epoch 515, loss: 1840316.875000
epoch 516, loss: 1838476.875000
epoch 517, loss: 1836639.000000
epoch 518, loss: 1834803.000000
epoch 519, loss: 1832968.625000
epoch 520, loss: 1831136.250000
epoch 521, loss: 1829305.500000
epoch 52

epoch 747, loss: 1459199.250000
epoch 748, loss: 1457740.500000
epoch 749, loss: 1456283.375000
epoch 750, loss: 1454827.375000
epoch 751, loss: 1453372.875000
epoch 752, loss: 1451920.000000
epoch 753, loss: 1450468.500000
epoch 754, loss: 1449018.375000
epoch 755, loss: 1447569.750000
epoch 756, loss: 1446122.625000
epoch 757, loss: 1444677.125000
epoch 758, loss: 1443232.625000
epoch 759, loss: 1441789.875000
epoch 760, loss: 1440348.625000
epoch 761, loss: 1438908.750000
epoch 762, loss: 1437470.125000
epoch 763, loss: 1436033.000000
epoch 764, loss: 1434597.625000
epoch 765, loss: 1433163.375000
epoch 766, loss: 1431730.500000
epoch 767, loss: 1430299.125000
epoch 768, loss: 1428869.375000
epoch 769, loss: 1427441.000000
epoch 770, loss: 1426013.875000
epoch 771, loss: 1424588.250000
epoch 772, loss: 1423164.125000
epoch 773, loss: 1421741.375000
epoch 774, loss: 1420320.000000
epoch 775, loss: 1418900.125000
epoch 776, loss: 1417481.625000
epoch 777, loss: 1416064.625000
epoch 77

In [65]:
print(rmse)

[10871.607766346315, 1505.1851908722176]


In [82]:
%%time
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, features, labels , batch_size=10000, num_epochs= 1000, lr = 0.00005)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[6] = sqrt(mean_squared_error(Y_test ,pred))
print("RMSE = ", rmse_list[6])

epoch 1, loss: 1996789.000000
epoch 2, loss: 1996589.000000
epoch 3, loss: 1996389.125000
epoch 4, loss: 1996189.125000
epoch 5, loss: 1995989.000000
epoch 6, loss: 1995789.125000
epoch 7, loss: 1995588.875000
epoch 8, loss: 1995388.500000
epoch 9, loss: 1995188.250000
epoch 10, loss: 1994987.750000
epoch 11, loss: 1994787.000000
epoch 12, loss: 1994585.875000
epoch 13, loss: 1994384.750000
epoch 14, loss: 1994183.125000
epoch 15, loss: 1993981.125000
epoch 16, loss: 1993778.375000
epoch 17, loss: 1993575.250000
epoch 18, loss: 1993371.000000
epoch 19, loss: 1993166.000000
epoch 20, loss: 1992959.625000
epoch 21, loss: 1992751.875000
epoch 22, loss: 1992542.250000
epoch 23, loss: 1992330.500000
epoch 24, loss: 1992115.875000
epoch 25, loss: 1991898.125000
epoch 26, loss: 1991676.125000
epoch 27, loss: 1991449.375000
epoch 28, loss: 1991216.750000
epoch 29, loss: 1990976.875000
epoch 30, loss: 1990728.125000
epoch 31, loss: 1990468.375000
epoch 32, loss: 1990195.375000
epoch 33, loss: 1

epoch 263, loss: 1835285.750000
epoch 264, loss: 1173442.875000
epoch 265, loss: 1432675.250000
epoch 266, loss: 1977841.500000
epoch 267, loss: 1957905.375000
epoch 268, loss: 1956751.625000
epoch 269, loss: 1955897.750000
epoch 270, loss: 1955191.375000
epoch 271, loss: 1954586.375000
epoch 272, loss: 1954100.625000
epoch 273, loss: 1953703.375000
epoch 274, loss: 1953374.250000
epoch 275, loss: 1953037.750000
epoch 276, loss: 1952480.625000
epoch 277, loss: 1950802.375000
epoch 278, loss: 1943797.375000
epoch 279, loss: 1911990.500000
epoch 280, loss: 1768441.500000
epoch 281, loss: 1206049.875000
epoch 282, loss: 569062.062500
epoch 283, loss: 686274.125000
epoch 284, loss: 1084307.375000
epoch 285, loss: 2007792.250000
epoch 286, loss: 1970578.875000
epoch 287, loss: 1966208.250000
epoch 288, loss: 1960618.750000
epoch 289, loss: 1957145.375000
epoch 290, loss: 1955311.375000
epoch 291, loss: 1953652.125000
epoch 292, loss: 1952239.875000
epoch 293, loss: 1951072.250000
epoch 294,

epoch 522, loss: 192797.000000
epoch 523, loss: 287026.375000
epoch 524, loss: 825731.437500
epoch 525, loss: 462123.875000
epoch 526, loss: 299434.437500
epoch 527, loss: 92106.132812
epoch 528, loss: 60777.503906
epoch 529, loss: 58741.375000
epoch 530, loss: 74461.015625
epoch 531, loss: 118259.570312
epoch 532, loss: 363280.562500
epoch 533, loss: 230553.218750
epoch 534, loss: 668993.625000
epoch 535, loss: 283217.062500
epoch 536, loss: 730999.750000
epoch 537, loss: 441466.968750
epoch 538, loss: 161361.281250
epoch 539, loss: 232265.312500
epoch 540, loss: 740587.187500
epoch 541, loss: 168138.859375
epoch 542, loss: 243519.046875
epoch 543, loss: 62872.792969
epoch 544, loss: 47374.664062
epoch 545, loss: 47428.796875
epoch 546, loss: 56129.175781
epoch 547, loss: 90542.070312
epoch 548, loss: 264159.875000
epoch 549, loss: 316322.125000
epoch 550, loss: 1021200.312500
epoch 551, loss: 149310.890625
epoch 552, loss: 180747.421875
epoch 553, loss: 544778.250000
epoch 554, loss:

epoch 791, loss: 19268.111328
epoch 792, loss: 27347.669922
epoch 793, loss: 18833.947266
epoch 794, loss: 26646.152344
epoch 795, loss: 18422.101562
epoch 796, loss: 25998.867188
epoch 797, loss: 18013.111328
epoch 798, loss: 25320.279297
epoch 799, loss: 17640.925781
epoch 800, loss: 24761.863281
epoch 801, loss: 17250.933594
epoch 802, loss: 24130.593750
epoch 803, loss: 16890.421875
epoch 804, loss: 23565.498047
epoch 805, loss: 16536.218750
epoch 806, loss: 23023.730469
epoch 807, loss: 16182.386719
epoch 808, loss: 22462.808594
epoch 809, loss: 15836.333008
epoch 810, loss: 21927.394531
epoch 811, loss: 15492.498047
epoch 812, loss: 21380.703125
epoch 813, loss: 15159.816406
epoch 814, loss: 20861.923828
epoch 815, loss: 14849.877930
epoch 816, loss: 20395.343750
epoch 817, loss: 14542.281250
epoch 818, loss: 19927.855469
epoch 819, loss: 14236.614258
epoch 820, loss: 19459.482422
epoch 821, loss: 13931.998047
epoch 822, loss: 18997.066406
epoch 823, loss: 13637.706055
epoch 824,

In [83]:
print("test RMSE = ", rmse_list[6])

test RMSE =  89.9516547598541


#### case 8

In [87]:
%%time
# tune
rmse = [[0,0,0], [0,0,0]]

Lr = [0.00005, 0.00001]
Wd = [0.2, 0.5, 0.8]
for i in range(len(Lr)):
    for j in range(len(Wd)):
        print(Lr[i], Wd[j])
        net = nn.Sequential()
        net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
        net.initialize(init.Normal(sigma=0.01))
        train(net, features, labels, batch_size=10000, num_epochs= 1000, lr = Lr[i], wd = Wd[j])
        pred = net(np.array(X_valid)).reshape(1,-1)[0]
        rmse[i][j] = sqrt(mean_squared_error(Y_valid ,pred))
        print("RMSE = ", rmse[i][j])

5e-05 0.2
epoch 1, loss: 1996789.000000
epoch 2, loss: 1996588.875000
epoch 3, loss: 1996388.750000
epoch 4, loss: 1996188.625000
epoch 5, loss: 1995988.375000
epoch 6, loss: 1995788.250000
epoch 7, loss: 1995587.750000
epoch 8, loss: 1995387.250000
epoch 9, loss: 1995186.375000
epoch 10, loss: 1994985.500000
epoch 11, loss: 1994784.125000
epoch 12, loss: 1994582.625000
epoch 13, loss: 1994380.750000
epoch 14, loss: 1994178.125000
epoch 15, loss: 1993975.000000
epoch 16, loss: 1993771.000000
epoch 17, loss: 1993566.000000
epoch 18, loss: 1993360.000000
epoch 19, loss: 1993152.500000
epoch 20, loss: 1992943.250000
epoch 21, loss: 1992731.625000
epoch 22, loss: 1992517.625000
epoch 23, loss: 1992300.500000
epoch 24, loss: 1992079.375000
epoch 25, loss: 1991853.625000
epoch 26, loss: 1991622.250000
epoch 27, loss: 1991383.625000
epoch 28, loss: 1991136.500000
epoch 29, loss: 1990879.000000
epoch 30, loss: 1990608.750000
epoch 31, loss: 1990323.000000
epoch 32, loss: 1990018.000000
epoch 3

epoch 263, loss: 237921.875000
epoch 264, loss: 88040.703125
epoch 265, loss: 80391.921875
epoch 266, loss: 98842.265625
epoch 267, loss: 233741.437500
epoch 268, loss: 510380.750000
epoch 269, loss: 1492891.500000
epoch 270, loss: 935429.625000
epoch 271, loss: 501747.468750
epoch 272, loss: 274361.125000
epoch 273, loss: 307929.500000
epoch 274, loss: 783266.000000
epoch 275, loss: 209284.078125
epoch 276, loss: 182901.875000
epoch 277, loss: 75059.921875
epoch 278, loss: 67144.085938
epoch 279, loss: 70177.750000
epoch 280, loss: 100415.703125
epoch 281, loss: 294591.187500
epoch 282, loss: 377275.593750
epoch 283, loss: 1218666.000000
epoch 284, loss: 453890.156250
epoch 285, loss: 186878.484375
epoch 286, loss: 341747.187500
epoch 287, loss: 390412.812500
epoch 288, loss: 971617.187500
epoch 289, loss: 793170.250000
epoch 290, loss: 959966.125000
epoch 291, loss: 251787.625000
epoch 292, loss: 136427.343750
epoch 293, loss: 100206.937500
epoch 294, loss: 97291.906250
epoch 295, lo

epoch 533, loss: 9112.291016
epoch 534, loss: 13891.577148
epoch 535, loss: 9496.524414
epoch 536, loss: 14469.689453
epoch 537, loss: 9703.847656
epoch 538, loss: 12336.803711
epoch 539, loss: 8049.443848
epoch 540, loss: 8838.971680
epoch 541, loss: 5980.654297
epoch 542, loss: 6536.281738
epoch 543, loss: 4904.065430
epoch 544, loss: 5513.634766
epoch 545, loss: 4560.494629
epoch 546, loss: 5283.356934
epoch 547, loss: 4679.724609
epoch 548, loss: 5595.078125
epoch 549, loss: 5178.594238
epoch 550, loss: 6410.162598
epoch 551, loss: 6059.273926
epoch 552, loss: 7761.090332
epoch 553, loss: 7298.236816
epoch 554, loss: 9580.408203
epoch 555, loss: 8732.776367
epoch 556, loss: 11549.114258
epoch 557, loss: 9954.939453
epoch 558, loss: 13001.809570
epoch 559, loss: 10452.677734
epoch 560, loss: 13278.516602
epoch 561, loss: 10036.037109
epoch 562, loss: 12345.456055
epoch 563, loss: 8993.761719
epoch 564, loss: 10800.440430
epoch 565, loss: 7786.529785
epoch 566, loss: 9293.943359
epoc

epoch 817, loss: 1173.307251
epoch 818, loss: 1281.713745
epoch 819, loss: 1111.210327
epoch 820, loss: 1218.856567
epoch 821, loss: 1039.509155
epoch 822, loss: 1145.285156
epoch 823, loss: 966.540405
epoch 824, loss: 1071.700439
epoch 825, loss: 895.321106
epoch 826, loss: 1000.575378
epoch 827, loss: 832.095154
epoch 828, loss: 938.796082
epoch 829, loss: 779.086060
epoch 830, loss: 889.997375
epoch 831, loss: 739.270020
epoch 832, loss: 852.354614
epoch 833, loss: 712.004028
epoch 834, loss: 822.982910
epoch 835, loss: 693.605225
epoch 836, loss: 798.612305
epoch 837, loss: 683.850586
epoch 838, loss: 774.585449
epoch 839, loss: 678.138306
epoch 840, loss: 749.887329
epoch 841, loss: 671.951233
epoch 842, loss: 722.954285
epoch 843, loss: 664.392090
epoch 844, loss: 692.994324
epoch 845, loss: 654.475586
epoch 846, loss: 664.436462
epoch 847, loss: 643.649902
epoch 848, loss: 638.161072
epoch 849, loss: 632.993774
epoch 850, loss: 614.733459
epoch 851, loss: 623.414429
epoch 852, l

epoch 98, loss: 942872.687500
epoch 99, loss: 1185716.500000
epoch 100, loss: 1503339.750000
epoch 101, loss: 1722652.500000
epoch 102, loss: 1630463.000000
epoch 103, loss: 1995688.750000
epoch 104, loss: 1979530.000000
epoch 105, loss: 1979246.125000
epoch 106, loss: 1978975.375000
epoch 107, loss: 1978566.250000
epoch 108, loss: 1977289.125000
epoch 109, loss: 1971028.000000
epoch 110, loss: 1935620.875000
epoch 111, loss: 1743486.000000
epoch 112, loss: 1093276.125000
epoch 113, loss: 921486.000000
epoch 114, loss: 1359913.000000
epoch 115, loss: 1084167.125000
epoch 116, loss: 2199821.750000
epoch 117, loss: 1590064.625000
epoch 118, loss: 925985.687500
epoch 119, loss: 842351.250000
epoch 120, loss: 862366.125000
epoch 121, loss: 1971144.500000
epoch 122, loss: 1916215.875000
epoch 123, loss: 1686107.500000
epoch 124, loss: 947311.312500
epoch 125, loss: 1064033.875000
epoch 126, loss: 1885548.375000
epoch 127, loss: 1521117.000000
epoch 128, loss: 996727.375000
epoch 129, loss: 

...

epoch 666, loss: 1094.729248
epoch 667, loss: 1006.939209
epoch 668, loss: 1083.840454
epoch 669, loss: 997.918396
epoch 670, loss: 1078.704468
epoch 671, loss: 993.006470
epoch 672, loss: 1074.119507
epoch 673, loss: 988.298828
epoch 674, loss: 1068.851196
epoch 675, loss: 984.484497
epoch 676, loss: 1062.063354
epoch 677, loss: 978.469421
epoch 678, loss: 1059.845459
epoch 679, loss: 976.148315
epoch 680, loss: 1056.942871
epoch 681, loss: 973.325073
epoch 682, loss: 1054.370483
epoch 683, loss: 970.334106
epoch 684, loss: 1050.773682
epoch 685, loss: 967.103699
epoch 686, loss: 1043.368652
epoch 687, loss: 960.702026
epoch 688, loss: 1036.829102
epoch 689, loss: 954.763306
epoch 690, loss: 1030.614136
epoch 691, loss: 949.960510
epoch 692, loss: 1025.334351
epoch 693, loss: 944.379822
epoch 694, loss: 1025.597534
epoch 695, loss: 944.162109
epoch 696, loss: 1017.888306
epoch 697, loss: 937.151611
epoch 698, loss: 1015.851501
epoch 699, loss: 935.282471
epoch 700, loss: 1008.705322
e

epoch 957, loss: 461.731659
epoch 958, loss: 495.641144
epoch 959, loss: 461.438110
epoch 960, loss: 490.712006
epoch 961, loss: 457.150085
epoch 962, loss: 490.035645
epoch 963, loss: 456.388641
epoch 964, loss: 485.253693
epoch 965, loss: 452.341644
epoch 966, loss: 481.928345
epoch 967, loss: 449.103302
epoch 968, loss: 480.633209
epoch 969, loss: 447.785858
epoch 970, loss: 479.317108
epoch 971, loss: 446.668243
epoch 972, loss: 477.242798
epoch 973, loss: 444.495636
epoch 974, loss: 473.794647
epoch 975, loss: 441.407715
epoch 976, loss: 471.882050
epoch 977, loss: 439.424103
epoch 978, loss: 465.972961
epoch 979, loss: 434.421204
epoch 980, loss: 462.790710
epoch 981, loss: 431.441559
epoch 982, loss: 459.691864
epoch 983, loss: 428.637146
epoch 984, loss: 456.467041
epoch 985, loss: 425.582153
epoch 986, loss: 453.631989
epoch 987, loss: 423.055786
epoch 988, loss: 450.994812
epoch 989, loss: 420.580139
epoch 990, loss: 449.312347
epoch 991, loss: 418.925812
epoch 992, loss: 446

epoch 221, loss: 1973883.500000
epoch 222, loss: 1972667.375000
epoch 223, loss: 1971291.750000
epoch 224, loss: 1969725.000000
epoch 225, loss: 1967927.875000
epoch 226, loss: 1965850.625000
epoch 227, loss: 1963430.250000
epoch 228, loss: 1960586.000000
epoch 229, loss: 1957213.000000
epoch 230, loss: 1953174.125000
epoch 231, loss: 1948288.375000
epoch 232, loss: 1942312.750000
epoch 233, loss: 1934918.625000
epoch 234, loss: 1925655.375000
epoch 235, loss: 1913893.500000
epoch 236, loss: 1898745.250000
epoch 237, loss: 1878937.375000
epoch 238, loss: 1852623.250000
epoch 239, loss: 1817092.875000
epoch 240, loss: 1768349.250000
epoch 241, loss: 1700549.750000
epoch 242, loss: 1605415.125000
epoch 243, loss: 1472206.125000
epoch 244, loss: 1290076.500000
epoch 245, loss: 1056812.625000
epoch 246, loss: 797040.562500
epoch 247, loss: 573325.187500
epoch 248, loss: 441567.187500
epoch 249, loss: 382396.656250
epoch 250, loss: 346528.343750
epoch 251, loss: 315742.656250
epoch 252, los

epoch 499, loss: 2059.995117
epoch 500, loss: 2237.176514
epoch 501, loss: 2044.049438
epoch 502, loss: 2210.156738
epoch 503, loss: 2019.583618
epoch 504, loss: 2192.801758
epoch 505, loss: 2002.317749
epoch 506, loss: 2170.733398
epoch 507, loss: 1982.151611
epoch 508, loss: 2153.440430
epoch 509, loss: 1964.427368
epoch 510, loss: 2124.443115
epoch 511, loss: 1938.768188
epoch 512, loss: 2100.374512
epoch 513, loss: 1915.072998
epoch 514, loss: 2078.802002
epoch 515, loss: 1893.331543
epoch 516, loss: 2045.979370
epoch 517, loss: 1863.713013
epoch 518, loss: 2013.243042
epoch 519, loss: 1834.256958
epoch 520, loss: 1983.887573
epoch 521, loss: 1808.981445
epoch 522, loss: 1956.206787
epoch 523, loss: 1784.684448
epoch 524, loss: 1928.769775
epoch 525, loss: 1759.070435
epoch 526, loss: 1899.489380
epoch 527, loss: 1733.536621
epoch 528, loss: 1867.479980
epoch 529, loss: 1707.564819
epoch 530, loss: 1862.871460
epoch 531, loss: 1700.276245
epoch 532, loss: 1843.487549
epoch 533, los

epoch 786, loss: 761.518250
epoch 787, loss: 704.190430
epoch 788, loss: 760.556274
epoch 789, loss: 703.068726
epoch 790, loss: 757.064270
epoch 791, loss: 700.305359
epoch 792, loss: 755.802002
epoch 793, loss: 698.463745
epoch 794, loss: 750.033264
epoch 795, loss: 693.219238
epoch 796, loss: 746.645508
epoch 797, loss: 690.076477
epoch 798, loss: 742.448730
epoch 799, loss: 686.612366
epoch 800, loss: 738.934814
epoch 801, loss: 683.327271
epoch 802, loss: 737.545227
epoch 803, loss: 681.673645
epoch 804, loss: 730.423828
epoch 805, loss: 675.557129
epoch 806, loss: 729.012634
epoch 807, loss: 673.664429
epoch 808, loss: 718.658325
epoch 809, loss: 664.609070
epoch 810, loss: 715.002075
epoch 811, loss: 660.911865
epoch 812, loss: 706.183228
epoch 813, loss: 653.189514
epoch 814, loss: 699.718506
epoch 815, loss: 647.239685
epoch 816, loss: 692.498779
epoch 817, loss: 641.578186
epoch 818, loss: 686.949158
epoch 819, loss: 636.166504
epoch 820, loss: 683.223206
epoch 821, loss: 633

In [88]:
print(rmse)

[[41.5854435240784, 64.01143675607811, 75.2612225025112], [38.214937042013126, 33.16731303251033, 33.35866119778753]]


In [91]:
%%time
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'), nn.Dense(45, activation='relu'), nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, features, labels, batch_size=10000, num_epochs= 1000, lr = 0.00001, wd =0.5)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[7] = sqrt(mean_squared_error(Y_test ,pred))
print("RMSE = ", rmse_list[7])

epoch 1, loss: 1996948.500000
epoch 2, loss: 1996908.500000
epoch 3, loss: 1996868.625000
epoch 4, loss: 1996828.500000
epoch 5, loss: 1996788.500000
epoch 6, loss: 1996748.625000
epoch 7, loss: 1996708.500000
epoch 8, loss: 1996668.500000
epoch 9, loss: 1996628.625000
epoch 10, loss: 1996588.500000
epoch 11, loss: 1996548.500000
epoch 12, loss: 1996508.375000
epoch 13, loss: 1996468.375000
epoch 14, loss: 1996428.500000
epoch 15, loss: 1996388.375000
epoch 16, loss: 1996348.375000
epoch 17, loss: 1996308.250000
epoch 18, loss: 1996268.375000
epoch 19, loss: 1996228.250000
epoch 20, loss: 1996188.250000
epoch 21, loss: 1996148.125000
epoch 22, loss: 1996108.125000
epoch 23, loss: 1996068.000000
epoch 24, loss: 1996027.875000
epoch 25, loss: 1995988.000000
epoch 26, loss: 1995947.875000
epoch 27, loss: 1995907.625000
epoch 28, loss: 1995867.750000
epoch 29, loss: 1995827.625000
epoch 30, loss: 1995787.500000
epoch 31, loss: 1995747.375000
epoch 32, loss: 1995707.375000
epoch 33, loss: 1

epoch 262, loss: 74413.289062
epoch 263, loss: 67608.851562
epoch 264, loss: 61619.617188
epoch 265, loss: 56354.328125
epoch 266, loss: 51711.664062
epoch 267, loss: 47590.371094
epoch 268, loss: 43926.718750
epoch 269, loss: 40657.976562
epoch 270, loss: 37715.136719
epoch 271, loss: 35056.214844
epoch 272, loss: 32639.058594
epoch 273, loss: 30444.050781
epoch 274, loss: 28445.898438
epoch 275, loss: 26617.785156
epoch 276, loss: 24944.472656
epoch 277, loss: 23408.173828
epoch 278, loss: 22001.376953
epoch 279, loss: 20709.507812
epoch 280, loss: 19511.880859
epoch 281, loss: 18401.617188
epoch 282, loss: 17375.683594
epoch 283, loss: 16427.445312
epoch 284, loss: 15542.311523
epoch 285, loss: 14716.604492
epoch 286, loss: 13943.418945
epoch 287, loss: 13219.670898
epoch 288, loss: 12541.037109
epoch 289, loss: 11899.489258
epoch 290, loss: 11300.887695
epoch 291, loss: 10739.122070
epoch 292, loss: 10210.003906
epoch 293, loss: 9713.778320
epoch 294, loss: 9245.217773
epoch 295, l

epoch 544, loss: 1622.850708
epoch 545, loss: 1757.247803
epoch 546, loss: 1607.512817
epoch 547, loss: 1741.338379
epoch 548, loss: 1592.634766
epoch 549, loss: 1711.128540
epoch 550, loss: 1568.479004
epoch 551, loss: 1697.418457
epoch 552, loss: 1554.827148
epoch 553, loss: 1683.203613
epoch 554, loss: 1542.778442
epoch 555, loss: 1666.329712
epoch 556, loss: 1528.249634
epoch 557, loss: 1659.442139
epoch 558, loss: 1522.176392
epoch 559, loss: 1649.266846
epoch 560, loss: 1514.472778
epoch 561, loss: 1643.566162
epoch 562, loss: 1507.885010
epoch 563, loss: 1632.998413
epoch 564, loss: 1498.558105
epoch 565, loss: 1629.846802
epoch 566, loss: 1494.340332
epoch 567, loss: 1622.878784
epoch 568, loss: 1487.265747
epoch 569, loss: 1611.803101
epoch 570, loss: 1476.635254
epoch 571, loss: 1599.495239
epoch 572, loss: 1465.100952
epoch 573, loss: 1594.415405
epoch 574, loss: 1459.088379
epoch 575, loss: 1572.152832
epoch 576, loss: 1440.799805
epoch 577, loss: 1556.004150
epoch 578, los

epoch 832, loss: 594.529602
epoch 833, loss: 640.075623
epoch 834, loss: 592.092468
epoch 835, loss: 634.735413
epoch 836, loss: 587.528076
epoch 837, loss: 627.352173
epoch 838, loss: 581.118530
epoch 839, loss: 625.641479
epoch 840, loss: 579.324219
epoch 841, loss: 623.878540
epoch 842, loss: 577.700745
epoch 843, loss: 623.158020
epoch 844, loss: 576.758423
epoch 845, loss: 620.069824
epoch 846, loss: 574.005920
epoch 847, loss: 615.572754
epoch 848, loss: 570.177246
epoch 849, loss: 612.510620
epoch 850, loss: 567.264832
epoch 851, loss: 609.428833
epoch 852, loss: 564.393738
epoch 853, loss: 606.724365
epoch 854, loss: 561.982178
epoch 855, loss: 608.043274
epoch 856, loss: 562.790771
epoch 857, loss: 605.665833
epoch 858, loss: 560.577393
epoch 859, loss: 599.980896
epoch 860, loss: 555.946228
epoch 861, loss: 593.924377
epoch 862, loss: 550.647949
epoch 863, loss: 591.612793
epoch 864, loss: 548.293579
epoch 865, loss: 595.318726
epoch 866, loss: 551.020447
epoch 867, loss: 590

In [92]:
print("test RMSE = ", rmse_list[7])

test RMSE =  41.055409506861025


#### case 9

In [95]:
%%time
# tune
rmse = [[0,0], [0,0]]

Lr = [0.00005, 0.00001]
Drop = [0.5, 0.7]
for i in range(len(Lr)):
    for j in range(len(Drop)):
        print(Lr[i], Drop[j])
        net = nn.Sequential()
        net.add(nn.Dense(45, activation='relu'),
                 nn.Dropout(Drop[j]),
                 nn.Dense(45, activation='relu'),
                 nn.Dropout(Drop[j]),
                 nn.Dense(1))
        net.initialize(init.Normal(sigma=0.01))
        train(net, features, labels, batch_size=10000, num_epochs= 1000, lr = Lr[i])
        pred = net(np.array(X_valid)).reshape(1,-1)[0]
        rmse[i][j] = sqrt(mean_squared_error(Y_valid ,pred))
        print("RMSE = ", rmse[i][j])

5e-05 0.5
epoch 1, loss: 1996789.375000
epoch 2, loss: 1996589.500000
epoch 3, loss: 1996389.625000
epoch 4, loss: 1996189.750000
epoch 5, loss: 1995989.750000
epoch 6, loss: 1995789.750000
epoch 7, loss: 1995589.875000
epoch 8, loss: 1995389.750000
epoch 9, loss: 1995189.500000
epoch 10, loss: 1994989.125000
epoch 11, loss: 1994788.625000
epoch 12, loss: 1994588.125000
epoch 13, loss: 1994387.250000
epoch 14, loss: 1994186.125000
epoch 15, loss: 1993984.875000
epoch 16, loss: 1993783.125000
epoch 17, loss: 1993580.750000
epoch 18, loss: 1993377.750000
epoch 19, loss: 1993174.250000
epoch 20, loss: 1992969.625000
epoch 21, loss: 1992764.000000
epoch 22, loss: 1992557.000000
epoch 23, loss: 1992348.250000
epoch 24, loss: 1992137.750000
epoch 25, loss: 1991924.500000
epoch 26, loss: 1991708.500000
epoch 27, loss: 1991488.875000
epoch 28, loss: 1991264.875000
epoch 29, loss: 1991035.250000
epoch 30, loss: 1990799.625000
epoch 31, loss: 1990555.875000
epoch 32, loss: 1990302.500000
epoch 3

epoch 265, loss: 559779.000000
epoch 266, loss: 441212.250000
epoch 267, loss: 1394060.125000
epoch 268, loss: 809336.750000
epoch 269, loss: 1209184.125000
epoch 270, loss: 464871.781250
epoch 271, loss: 628137.750000
epoch 272, loss: 108341.257812
epoch 273, loss: 668860.000000
epoch 274, loss: 113608.617188
epoch 275, loss: 122398.656250
epoch 276, loss: 126625.398438
epoch 277, loss: 54471.109375
epoch 278, loss: 215383.328125
epoch 279, loss: 32641.939453
epoch 280, loss: 269650.500000
epoch 281, loss: 43000.113281
epoch 282, loss: 429806.125000
epoch 283, loss: 57560.101562
epoch 284, loss: 118702.195312
epoch 285, loss: 66422.500000
epoch 286, loss: 40616.164062
epoch 287, loss: 90938.382812
epoch 288, loss: 29044.517578
epoch 289, loss: 169442.296875
epoch 290, loss: 23421.708984
epoch 291, loss: 262413.187500
epoch 292, loss: 26087.398438
epoch 293, loss: 217637.296875
epoch 294, loss: 33698.843750
epoch 295, loss: 250579.359375
epoch 296, loss: 58450.343750
epoch 297, loss: 8

epoch 541, loss: 5761.221191
epoch 542, loss: 19500.619141
epoch 543, loss: 4615.669922
epoch 544, loss: 23755.187500
epoch 545, loss: 2658.935059
epoch 546, loss: 23076.267578
epoch 547, loss: 2797.632568
epoch 548, loss: 25685.302734
epoch 549, loss: 3658.672852
epoch 550, loss: 19794.769531
epoch 551, loss: 6816.491211
epoch 552, loss: 14930.972656
epoch 553, loss: 8939.804688
epoch 554, loss: 11496.481445
epoch 555, loss: 8868.125977
epoch 556, loss: 12827.798828
epoch 557, loss: 10456.628906
epoch 558, loss: 9663.197266
epoch 559, loss: 11783.146484
epoch 560, loss: 8968.666992
epoch 561, loss: 12608.406250
epoch 562, loss: 7313.148926
epoch 563, loss: 13718.708984
epoch 564, loss: 6302.741211
epoch 565, loss: 17492.884766
epoch 566, loss: 5035.432617
epoch 567, loss: 18245.500000
epoch 568, loss: 6466.266602
epoch 569, loss: 15407.170898
epoch 570, loss: 8758.949219
epoch 571, loss: 12209.819336
epoch 572, loss: 9960.926758
epoch 573, loss: 12639.178711
epoch 574, loss: 8598.1796

epoch 819, loss: 10276.868164
epoch 820, loss: 10073.500977
epoch 821, loss: 8110.930176
epoch 822, loss: 11250.553711
epoch 823, loss: 9502.005859
epoch 824, loss: 13279.038086
epoch 825, loss: 8964.083984
epoch 826, loss: 9543.444336
epoch 827, loss: 12036.359375
epoch 828, loss: 8388.451172
epoch 829, loss: 10317.614258
epoch 830, loss: 13272.113281
epoch 831, loss: 6132.526367
epoch 832, loss: 16201.148438
epoch 833, loss: 7021.207031
epoch 834, loss: 11985.584961
epoch 835, loss: 9315.333984
epoch 836, loss: 13905.769531
epoch 837, loss: 7380.687012
epoch 838, loss: 14934.153320
epoch 839, loss: 6956.083984
epoch 840, loss: 10977.067383
epoch 841, loss: 9054.025391
epoch 842, loss: 12347.758789
epoch 843, loss: 8342.377930
epoch 844, loss: 11399.750000
epoch 845, loss: 9965.640625
epoch 846, loss: 11648.672852
epoch 847, loss: 9953.817383
epoch 848, loss: 12075.179688
epoch 849, loss: 7483.390625
epoch 850, loss: 12851.649414
epoch 851, loss: 8927.578125
epoch 852, loss: 11222.235

epoch 92, loss: 778475.250000
epoch 93, loss: 272319.656250
epoch 94, loss: 954051.875000
epoch 95, loss: 346958.718750
epoch 96, loss: 100294.992188
epoch 97, loss: 1023628.500000
epoch 98, loss: 182045.656250
epoch 99, loss: 1085176.125000
epoch 100, loss: 848644.687500
epoch 101, loss: 1986228.375000
epoch 102, loss: 1983227.750000
epoch 103, loss: 1979906.875000
epoch 104, loss: 1965410.750000
epoch 105, loss: 1888804.875000
epoch 106, loss: 1548679.500000
epoch 107, loss: 1086717.750000
epoch 108, loss: 1310008.000000
epoch 109, loss: 380646.906250
epoch 110, loss: 1951412.250000
epoch 111, loss: 1821798.375000
epoch 112, loss: 1380788.875000
epoch 113, loss: 470833.250000
epoch 114, loss: 1373038.250000
epoch 115, loss: 448464.937500
epoch 116, loss: 1266781.000000
epoch 117, loss: 373470.093750
epoch 118, loss: 1002083.125000
epoch 119, loss: 233101.421875
epoch 120, loss: 1391969.250000
epoch 121, loss: 985815.375000
epoch 122, loss: 1014022.562500
epoch 123, loss: 126112.82031

...

epoch 718, loss: 4935.276367
epoch 719, loss: 4916.867188
epoch 720, loss: 4208.094727
epoch 721, loss: 4996.999512
epoch 722, loss: 5492.923828
epoch 723, loss: 5061.755859
epoch 724, loss: 5059.506836
epoch 725, loss: 4841.525391
epoch 726, loss: 4517.812988
epoch 727, loss: 4899.992188
epoch 728, loss: 5275.739258
epoch 729, loss: 4718.318359
epoch 730, loss: 5131.764160
epoch 731, loss: 5178.089844
epoch 732, loss: 5044.065918
epoch 733, loss: 4750.277832
epoch 734, loss: 4685.958496
epoch 735, loss: 4384.542480
epoch 736, loss: 4889.540039
epoch 737, loss: 4850.767090
epoch 738, loss: 4573.863770
epoch 739, loss: 4429.102051
epoch 740, loss: 4263.749512
epoch 741, loss: 4648.730469
epoch 742, loss: 4992.039062
epoch 743, loss: 5298.113770
epoch 744, loss: 5225.283691
epoch 745, loss: 4748.085938
epoch 746, loss: 4969.076172
epoch 747, loss: 4991.514648
epoch 748, loss: 5566.496582
epoch 749, loss: 4522.627441
epoch 750, loss: 5110.076172
epoch 751, loss: 5120.397949
epoch 752, los

RMSE =  95.78763285465651
1e-05 0.7
epoch 1, loss: 1996948.875000
epoch 2, loss: 1996908.750000
epoch 3, loss: 1996868.875000
epoch 4, loss: 1996828.875000
epoch 5, loss: 1996788.750000
epoch 6, loss: 1996748.750000
epoch 7, loss: 1996708.875000
epoch 8, loss: 1996668.750000
epoch 9, loss: 1996628.750000
epoch 10, loss: 1996588.625000
epoch 11, loss: 1996548.750000
epoch 12, loss: 1996508.750000
epoch 13, loss: 1996468.625000
epoch 14, loss: 1996428.750000
epoch 15, loss: 1996388.500000
epoch 16, loss: 1996348.625000
epoch 17, loss: 1996308.500000
epoch 18, loss: 1996268.500000
epoch 19, loss: 1996228.375000
epoch 20, loss: 1996188.500000
epoch 21, loss: 1996148.375000
epoch 22, loss: 1996108.375000
epoch 23, loss: 1996068.250000
epoch 24, loss: 1996028.375000
epoch 25, loss: 1995988.125000
epoch 26, loss: 1995948.250000
epoch 27, loss: 1995908.125000
epoch 28, loss: 1995868.000000
epoch 29, loss: 1995828.000000
epoch 30, loss: 1995787.875000
epoch 31, loss: 1995747.750000
epoch 32, lo

epoch 260, loss: 222122.156250
epoch 261, loss: 206390.093750
epoch 262, loss: 195249.875000
epoch 263, loss: 181186.609375
epoch 264, loss: 166345.781250
epoch 265, loss: 158332.281250
epoch 266, loss: 151641.234375
epoch 267, loss: 144593.281250
epoch 268, loss: 132221.703125
epoch 269, loss: 125909.718750
epoch 270, loss: 119974.218750
epoch 271, loss: 117309.054688
epoch 272, loss: 109534.953125
epoch 273, loss: 106234.421875
epoch 274, loss: 100814.570312
epoch 275, loss: 97088.101562
epoch 276, loss: 88432.773438
epoch 277, loss: 85139.453125
epoch 278, loss: 85047.531250
epoch 279, loss: 77397.890625
epoch 280, loss: 76265.578125
epoch 281, loss: 74556.265625
epoch 282, loss: 75381.437500
epoch 283, loss: 73355.539062
epoch 284, loss: 69468.968750
epoch 285, loss: 67438.695312
epoch 286, loss: 66089.507812
epoch 287, loss: 66273.414062
epoch 288, loss: 60758.687500
epoch 289, loss: 59112.050781
epoch 290, loss: 60887.656250
epoch 291, loss: 58988.964844
epoch 292, loss: 57346.44

epoch 533, loss: 24296.265625
epoch 534, loss: 24670.660156
epoch 535, loss: 24313.785156
epoch 536, loss: 23529.230469
epoch 537, loss: 23140.914062
epoch 538, loss: 21487.753906
epoch 539, loss: 21213.642578
epoch 540, loss: 19756.171875
epoch 541, loss: 21209.376953
epoch 542, loss: 21163.962891
epoch 543, loss: 19318.619141
epoch 544, loss: 21921.042969
epoch 545, loss: 19770.964844
epoch 546, loss: 20983.513672
epoch 547, loss: 23708.074219
epoch 548, loss: 22579.968750
epoch 549, loss: 21474.193359
epoch 550, loss: 22377.677734
epoch 551, loss: 21741.447266
epoch 552, loss: 21332.892578
epoch 553, loss: 20072.566406
epoch 554, loss: 21445.589844
epoch 555, loss: 22693.175781
epoch 556, loss: 21898.136719
epoch 557, loss: 22088.503906
epoch 558, loss: 23918.009766
epoch 559, loss: 21343.314453
epoch 560, loss: 21281.066406
epoch 561, loss: 20132.460938
epoch 562, loss: 19573.367188
epoch 563, loss: 21248.294922
epoch 564, loss: 19750.400391
epoch 565, loss: 20103.443359
epoch 566,

epoch 807, loss: 17066.546875
epoch 808, loss: 17418.027344
epoch 809, loss: 17750.693359
epoch 810, loss: 16380.267578
epoch 811, loss: 19320.058594
epoch 812, loss: 18690.269531
epoch 813, loss: 18250.261719
epoch 814, loss: 16418.187500
epoch 815, loss: 15722.115234
epoch 816, loss: 17970.408203
epoch 817, loss: 16635.171875
epoch 818, loss: 17111.505859
epoch 819, loss: 17640.919922
epoch 820, loss: 17041.298828
epoch 821, loss: 18350.037109
epoch 822, loss: 17562.332031
epoch 823, loss: 16996.830078
epoch 824, loss: 16933.945312
epoch 825, loss: 17114.328125
epoch 826, loss: 17481.638672
epoch 827, loss: 16666.234375
epoch 828, loss: 16102.729492
epoch 829, loss: 18188.330078
epoch 830, loss: 17999.037109
epoch 831, loss: 19254.111328
epoch 832, loss: 19351.146484
epoch 833, loss: 19261.437500
epoch 834, loss: 18042.808594
epoch 835, loss: 17986.957031
epoch 836, loss: 17583.458984
epoch 837, loss: 17386.314453
epoch 838, loss: 18726.388672
epoch 839, loss: 16617.070312
epoch 840,

In [96]:
print(rmse)

[[128.07132814219494, 251.4455767935212], [95.78763285465651, 181.76428900053634]]


In [142]:
%%time
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'),
         nn.Dropout(0.5),
         nn.Dense(45, activation='relu'),
         nn.Dropout(0.5),
         nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train(net, features, labels, batch_size=10000, num_epochs= 1000, lr = 0.00001)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[8] = sqrt(mean_squared_error(Y_test ,pred))
print("test RMSE = ", rmse_list[8])

epoch 1, loss: 1996949.125000
epoch 2, loss: 1996909.125000
epoch 3, loss: 1996869.000000
epoch 4, loss: 1996829.125000
epoch 5, loss: 1996789.000000
epoch 6, loss: 1996749.000000
epoch 7, loss: 1996709.125000
epoch 8, loss: 1996668.875000
epoch 9, loss: 1996629.000000
epoch 10, loss: 1996589.000000
epoch 11, loss: 1996548.875000
epoch 12, loss: 1996509.000000
epoch 13, loss: 1996468.875000
epoch 14, loss: 1996428.875000
epoch 15, loss: 1996389.000000
epoch 16, loss: 1996348.875000
epoch 17, loss: 1996308.875000
epoch 18, loss: 1996268.750000
epoch 19, loss: 1996228.875000
epoch 20, loss: 1996188.625000
epoch 21, loss: 1996148.750000
epoch 22, loss: 1996108.625000
epoch 23, loss: 1996068.625000
epoch 24, loss: 1996028.500000
epoch 25, loss: 1995988.625000
epoch 26, loss: 1995948.500000
epoch 27, loss: 1995908.500000
epoch 28, loss: 1995868.375000
epoch 29, loss: 1995828.375000
epoch 30, loss: 1995788.250000
epoch 31, loss: 1995748.125000
epoch 32, loss: 1995708.250000
epoch 33, loss: 1

epoch 261, loss: 255753.671875
epoch 262, loss: 230128.312500
epoch 263, loss: 212045.343750
epoch 264, loss: 194269.281250
epoch 265, loss: 180250.265625
epoch 266, loss: 167332.625000
epoch 267, loss: 151040.359375
epoch 268, loss: 141054.609375
epoch 269, loss: 129880.906250
epoch 270, loss: 118645.414062
epoch 271, loss: 110344.015625
epoch 272, loss: 104282.250000
epoch 273, loss: 96671.101562
epoch 274, loss: 89997.468750
epoch 275, loss: 86340.843750
epoch 276, loss: 79129.828125
epoch 277, loss: 73151.031250
epoch 278, loss: 69363.507812
epoch 279, loss: 65165.523438
epoch 280, loss: 62060.511719
epoch 281, loss: 58276.292969
epoch 282, loss: 57224.468750
epoch 283, loss: 53305.507812
epoch 284, loss: 49285.988281
epoch 285, loss: 46707.468750
epoch 286, loss: 46700.769531
epoch 287, loss: 43500.386719
epoch 288, loss: 42726.066406
epoch 289, loss: 38003.175781
epoch 290, loss: 36684.453125
epoch 291, loss: 36961.941406
epoch 292, loss: 33914.734375
epoch 293, loss: 33155.27734

epoch 536, loss: 7544.450195
epoch 537, loss: 7798.606445
epoch 538, loss: 7591.700195
epoch 539, loss: 6944.856934
epoch 540, loss: 7092.041504
epoch 541, loss: 7851.464844
epoch 542, loss: 6993.124023
epoch 543, loss: 8873.675781
epoch 544, loss: 8869.087891
epoch 545, loss: 8025.462402
epoch 546, loss: 8439.941406
epoch 547, loss: 6575.213379
epoch 548, loss: 7060.840820
epoch 549, loss: 6817.047852
epoch 550, loss: 7996.435059
epoch 551, loss: 6866.510254
epoch 552, loss: 8072.176758
epoch 553, loss: 9018.693359
epoch 554, loss: 8836.496094
epoch 555, loss: 8847.632812
epoch 556, loss: 8874.811523
epoch 557, loss: 8653.359375
epoch 558, loss: 9783.688477
epoch 559, loss: 9396.834961
epoch 560, loss: 9753.106445
epoch 561, loss: 10553.688477
epoch 562, loss: 10986.395508
epoch 563, loss: 9817.424805
epoch 564, loss: 9174.624023
epoch 565, loss: 9659.918945
epoch 566, loss: 8841.100586
epoch 567, loss: 9263.875000
epoch 568, loss: 8282.307617
epoch 569, loss: 7727.131348
epoch 570, l

epoch 819, loss: 5727.375977
epoch 820, loss: 4997.534180
epoch 821, loss: 5256.454102
epoch 822, loss: 5092.500977
epoch 823, loss: 4920.309082
epoch 824, loss: 4888.558594
epoch 825, loss: 5032.578125
epoch 826, loss: 5423.187988
epoch 827, loss: 5070.243164
epoch 828, loss: 5595.957520
epoch 829, loss: 5074.380859
epoch 830, loss: 4553.700195
epoch 831, loss: 4725.984863
epoch 832, loss: 5206.955078
epoch 833, loss: 4657.532227
epoch 834, loss: 4987.408203
epoch 835, loss: 5197.547363
epoch 836, loss: 5038.368164
epoch 837, loss: 5302.613770
epoch 838, loss: 4888.034668
epoch 839, loss: 5220.292969
epoch 840, loss: 5049.235840
epoch 841, loss: 4812.297363
epoch 842, loss: 4776.279785
epoch 843, loss: 4982.997559
epoch 844, loss: 5357.195801
epoch 845, loss: 4848.423828
epoch 846, loss: 4990.192871
epoch 847, loss: 4843.212402
epoch 848, loss: 5397.586426
epoch 849, loss: 5665.743652
epoch 850, loss: 5205.951172
epoch 851, loss: 5890.054199
epoch 852, loss: 5679.780762
epoch 853, los

In [143]:
print("test RMSE = ", rmse_list[8])

test RMSE =  101.03835365572799


#### case 10

In [109]:
def train_nonstop(net, features, labels, batch_size, num_epochs, lr=0.01, wd=0): # weight decay
    data_iter = load_array((features, labels), batch_size)
    loss = gloss.L2Loss()
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd})

    loss_record = [0] * num_epochs
    # net = net.cast('float64')
    for epoch in range(1, num_epochs + 1):
        for X, y in data_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        l = loss(net(features), labels)
        print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

In [114]:
%%time
#tune
rmse = [[0,0], [0,0]]

Lr = [0.1, 0.3]
Drop = [0.2, 0.5]
for i in range(len(Lr)):
    for j in range(len(Drop)):
        print(Lr[i], Drop[j])
        net = nn.Sequential()
        net.add(nn.Dense(45, activation='relu'),
                 nn.Dropout(Drop[j]),
                 nn.Dense(45, activation='relu'),
                 nn.Dropout(Drop[j]),
                 nn.Dense(1))
        net.initialize(init.Normal(sigma=0.01))
        train_nonstop(net, X_train, de_Y_train, batch_size=len(X_train), num_epochs= 50, lr = Lr[i])
        pred = net(np.array(X_valid)).reshape(1,-1)[0]
        rmse[i][j] = sqrt(mean_squared_error(de_Y_valid ,pred))
        print("RMSE = ", rmse[i][j])

0.1 0.2
epoch 1, loss: 59.725460
epoch 2, loss: 59.725426
epoch 3, loss: 59.725388
epoch 4, loss: 59.725353
epoch 5, loss: 59.725323
epoch 6, loss: 59.725292
epoch 7, loss: 59.725262
epoch 8, loss: 59.725227
epoch 9, loss: 59.725193
epoch 10, loss: 59.725166
epoch 11, loss: 59.725128
epoch 12, loss: 59.725094
epoch 13, loss: 59.725056
epoch 14, loss: 59.725018
epoch 15, loss: 59.724976
epoch 16, loss: 59.724930
epoch 17, loss: 59.724884
epoch 18, loss: 59.724831
epoch 19, loss: 59.724773
epoch 20, loss: 59.724716
epoch 21, loss: 59.724648
epoch 22, loss: 59.724575
epoch 23, loss: 59.724495
epoch 24, loss: 59.724407
epoch 25, loss: 59.724319
epoch 26, loss: 59.724213
epoch 27, loss: 59.724098
epoch 28, loss: 59.723972
epoch 29, loss: 59.723835
epoch 30, loss: 59.723675
epoch 31, loss: 59.723503
epoch 32, loss: 59.723312
epoch 33, loss: 59.723095
epoch 34, loss: 59.722855
epoch 35, loss: 59.722584
epoch 36, loss: 59.722275
epoch 37, loss: 59.721928
epoch 38, loss: 59.721527
epoch 39, los

In [115]:
print(rmse)

[[11.031118595199958, 11.027657440297958], [11.381700118469372, 11.035778249424476]]


In [116]:
%%time
net = nn.Sequential()
net.add(nn.Dense(45, activation='relu'),
         nn.Dropout(0.5),
         nn.Dense(45, activation='relu'),
         nn.Dropout(0.5),
         nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))
train_nonstop(net, X_train, de_Y_train, batch_size=len(X_train), num_epochs= 50, lr = 0.1)
pred = net(np.array(X_test)).reshape(1,-1)[0]
rmse_list[9] = sqrt(mean_squared_error(de_Y_test ,pred))
print("test RMSE = ", rmse_list[9])

epoch 1, loss: 59.725464
epoch 2, loss: 59.725407
epoch 3, loss: 59.725349
epoch 4, loss: 59.725285
epoch 5, loss: 59.725224
epoch 6, loss: 59.725163
epoch 7, loss: 59.725098
epoch 8, loss: 59.725033
epoch 9, loss: 59.724964
epoch 10, loss: 59.724899
epoch 11, loss: 59.724831
epoch 12, loss: 59.724754
epoch 13, loss: 59.724678
epoch 14, loss: 59.724590
epoch 15, loss: 59.724506
epoch 16, loss: 59.724407
epoch 17, loss: 59.724304
epoch 18, loss: 59.724190
epoch 19, loss: 59.724064
epoch 20, loss: 59.723930
epoch 21, loss: 59.723782
epoch 22, loss: 59.723610
epoch 23, loss: 59.723427
epoch 24, loss: 59.723225
epoch 25, loss: 59.722996
epoch 26, loss: 59.722740
epoch 27, loss: 59.722454
epoch 28, loss: 59.722134
epoch 29, loss: 59.721767
epoch 30, loss: 59.721352
epoch 31, loss: 59.720882
epoch 32, loss: 59.720341
epoch 33, loss: 59.719723
epoch 34, loss: 59.718998
epoch 35, loss: 59.718170
epoch 36, loss: 59.717194
epoch 37, loss: 59.716034
epoch 38, loss: 59.714672
epoch 39, loss: 59.71

### Q2

In [126]:
[1,11]

[1, 11]

In [144]:
import pandas as pd
case = ['OLS', 'MLP_0_dm', 'MLP_1_dm', 'MLP_2_dm', 'MLP_2_dm_L2', 'MLP_2_dm_dropout', 'MLP_2_ykeep', 'MLP_2_ykeep_L2', 'MLP_2_ykeep_dropout', 'MLP_2_dm_dropout_full']
table = {"case_index": list(range(1,11)),
         "case" : case,
         "test RMSE" : rmse_list}
result = pd.DataFrame(table)
result

,case_index,case,test RMSE
0,1,OLS,9.570046
1,2,MLP_0_dm,9.510605
2,3,MLP_1_dm,9.304589
3,4,MLP_2_dm,9.347290
4,5,MLP_2_dm_L2,9.899236
5,6,MLP_2_dm_dropout,9.360248
6,7,MLP_2_ykeep,89.951655
7,8,MLP_2_ykeep_L2,41.055410
8,9,MLP_2_ykeep_dropout,101.038354
9,10,MLP_2_dm_dropout_full,10.843488


In [123]:
# case 7 ~ 9 are keep y, without subtracting all target values in training and test set by the mean of the target values in the training set.
# the test RMSE is higher than other